In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 15.6 MB/s eta 0:00:00a 0:00:01


In [54]:
!pip install deep_sort_realtime ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.5 MB/s eta 0:00:00:00:010:01
^C
ERROR: Operation cancelled by user


In [ ]:
!git clone https://github.com/NirAharon/BoT-SORT.git
!git clone https://github.com/mikel-brostrom/Yolov8_DeepSORT_OSNet.git


Cloning into 'BoT-SORT'...
remote: Enumerating objects: 1013, done.
remote: Total 1013 (delta 0), reused 0 (delta 0), pack-reused 1013 (from 1)
Receiving objects: 100% (1013/1013), 55.77 MiB | 53.12 MiB/s, done.
Resolving deltas: 100% (286/286), done.
Cloning into 'Yolov8_DeepSORT_OSNet'...
Username for 'https://github.com': 

## Fine Tune

In [ ]:
from ultralytics import YOLO


model = YOLO("yolov8m.pt")


model.train(
    data="/kaggle/input/trackingfull/trackingfull/fawry.yaml",  
    epochs=50,          
    imgsz=704,            
    batch=8,             
    workers=2,           
    optimizer="Adam",     
    lr0=0.0001,           
    dropout=0.1,          
    augment=True,         
    hsv_h=0.015,          
    hsv_s=0.7,            
    hsv_v=0.4,            
    flipud=0.3,           
    fliplr=0.5,           
    mosaic=0.5,           
    mixup=0.0,            
    degrees=5,            
    translate=0.1,        
    scale=0.5,            
    shear=2,              
    perspective=0.001,
    save_period=10,
)


## Testing

In [ ]:
import torch
import os
import pandas as pd
import cv2
from ultralytics import YOLO  # Assuming you used YOLO for tracking

# Load your trained model
model_path = "/kaggle/input/model-trial/pytorch/default/1/best (2).pt"  # Update with your model path
tracking_model = YOLO(model_path)  

# Path to test video frames
test_frames_dir = "/kaggle/input/tracking/tracking/tracking/test/01/img1"  # Update with your test frames directory
output_csv = "/kaggle/working/tracking_results.csv"

# Get list of image frames
frame_files = sorted([f for f in os.listdir(test_frames_dir) if f.endswith(".jpg")])
total_frames = len(frame_files)

# Initialize an empty list to store tracking results
tracking_data = []

# Process each test frame
for frame_idx, frame_name in enumerate(frame_files, start=1):  
    frame_path = os.path.join(test_frames_dir, frame_name)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Warning: Could not load frame {frame_path}. Skipping.")
        continue  # Skip this frame

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for YOLO

    # Run model inference for tracking
    results = tracking_model.track(frame, persist=False)  # Disable persist to avoid tracking errors

    for result in results:
        if result.boxes:
            for box in result.boxes:
                x, y, w, h = map(int, box.xywh[0])  # Bounding box (x, y, width, height)
                confidence = float(box.conf[0])  # Confidence score
                track_id = int(box.id[0]) if box.id is not None else -1  # Handle missing track IDs

                tracking_data.append({
                    "frame": frame_idx,
                    "track_id": track_id,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h,
                    "confidence": confidence
                })

# Save results to CSV
df = pd.DataFrame(tracking_data)
df.to_csv(output_csv, index=False)
print(f"Tracking results saved to {output_csv}")


In [ ]:
tracking_model.val()

In [ ]:

tracking_model.train(data="/kaggle/input/trackingfinal/trackingfinal/fawry.yaml",
    epochs=20,
    imgsz=704,
    batch=8,
    workers=2,
    save_period=10)


In [ ]:
from ultralytics import YOLO
model=YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")
model.train(epochs=20,
    imgsz=704,
    batch=8,
    workers=2,
    save_period=10,
           resume=True)

In [ ]:
model2=YOLO("/kaggle/input/model-trial/pytorch/default/3/best (2).pt")


In [ ]:
model2.train(epochs=1,
    imgsz=704,
    batch=8,
    workers=2,
    save_period=10)

In [ ]:
model3=YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
image_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1/000051.jpg"  # Replace with your test image path
image = cv2.imread(image_path)
model2=YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")
# Resize the image to 704x704 while keeping aspect ratio
image_resized = cv2.resize(image, (704, 704))

# Run YOLO inference on the resized image
results = model2(image_resized)

# Convert image to RGB for display
image_resized_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)

# Iterate through detections
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
        conf = float(box.conf[0])  # Confidence score
        cls = int(box.cls[0])  # Class ID
        label = f"{model2.names[cls]}: {conf:.2f}"

        # Draw bounding box and label
        cv2.rectangle(image_resized_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image_resized_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with bounding boxes
plt.imshow(image_resized_rgb)
plt.axis("off")
plt.show()


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd

# Reload the tracking results
output_csv = "/kaggle/working/tracking_results.csv"
df = pd.read_csv(output_csv)

# Directory containing the frames
test_frames_dir = "/kaggle/input/tracking/tracking/tracking/test/01/img1"  # Update if needed

# Select a frame to visualize
frame_to_show = df["frame"].iloc[0]  # Get the first tracked frame
frame_path = os.path.join(test_frames_dir, f"{frame_to_show:06d}.jpg")  # Adjust if needed
frame = cv2.imread(frame_path)

if frame is None:
    print(f"Error loading frame {frame_path}")
else:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get bounding boxes for the selected frame
    detections = df[df["frame"] == frame_to_show]

    for _, row in detections.iterrows():
        x, y, w, h = int(row["x"]), int(row["y"]), int(row["width"]), int(row["height"])
        track_id = int(row["track_id"])
        
        # Draw bounding box (fixed)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Show image
    plt.figure(figsize=(10, 6))
    plt.imshow(frame)
    plt.axis("off")
    plt.show()


In [ ]:
import cv2
import os
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load your trained model
model_path = "/kaggle/input/model-trial/pytorch/default/1/best (2).pt"  # Update with your model path
tracking_model = YOLO(model_path)

# Input frame
frame_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1/000016.jpg"  # Update with your image path
frame = cv2.imread(frame_path)

if frame is None:
    print(f"Error: Could not load frame {frame_path}")
else:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for YOLO

    # Run tracking on the frame
    results = tracking_model.track(frame, persist=False)  # Disable persist to avoid errors

    # Draw bounding boxes
    for result in results:
        if result.boxes:
            for box in result.boxes:
                x, y, w, h = map(int, box.xywh[0])  # Convert bounding box to int
                confidence = float(box.conf[0])  # Confidence score
                track_id = int(box.id[0]) if box.id is not None else -1  # Assign track ID
                
                # Draw bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Convert back to BGR for saving (OpenCV saves in BGR format)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Save the output image
    output_path = "/kaggle/working/tracked_frame.jpg"  # Save output image
    cv2.imwrite(output_path, frame)

    # Display the image
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    print(f"✅ Tracking results saved to: {output_path}")


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Define image and label paths
image_folder = "/kaggle/input/tracking/tracking/tracking/train/02/img1/"  # Update with your actual folder
label_path = "/kaggle/input/gtof02/gt.txt"  # Update with actual label path

# Define frame of interest
frame_of_interest = 1478  # Update this with your desired frame

# Generate corresponding image filename
image_filename = f"{frame_of_interest:06d}.jpg"  # Converts 1312 to "001312.jpg"
image_path = os.path.join(image_folder, image_filename)

# Check if image exists
if not os.path.exists(image_path):
    print(f"Image for frame {frame_of_interest} not found: {image_path}")
else:
    print(f"Image for frame {frame_of_interest} found: {image_path}")

    # Load and display the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 8))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Frame {frame_of_interest}")
    plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# Define image and label paths (update accordingly)
image_path = "/kaggle/input/tracking/tracking/tracking/train/02/img1/000025.jpg"  # Update this with your actual image path
label_path = "/kaggle/input/gtof02/gt.txt"  # Update this with your actual label path

# Extract frame number from image filename
image_filename = os.path.basename(image_path)
frame_of_interest = int(image_filename.split('.')[0])  # Convert to integer

# Load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (since OpenCV loads in BGR format)
height, width, _ = image.shape

# Read MOT label file and draw bounding boxes
unique_classes = set()

with open(label_path, "r") as file:
    lines = file.readlines()
    for line in lines:
        data = line.strip().split(',')  # Split by comma since MOT format is CSV-like
        frame_num = int(data[0])
        
        if frame_num == frame_of_interest:  # Process only the required frame
            track_id = int(data[1])
            x1 = int(float(data[2]))  # X (top-left corner)
            y1 = int(float(data[3]))  # Y (top-left corner)
            w = int(float(data[4]))   # Width
            h = int(float(data[5]))   # Height
            confidence = float(data[6])  # Confidence score
            class_id = int(data[7])  # Class ID from the file
            visibility = float(data[8])  # Visibility score
            
            unique_classes.add(class_id)
            
            x2 = x1 + w
            y2 = y1 + h
            
            # Draw bounding box
            color = (255, 0, 0)  # Red color for bounding box
            thickness = 2
            cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
            
            # Put class label
            label_text = f"Class: {class_id} (ID: {track_id})"
            cv2.putText(image, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

# Print unique class IDs
print("Unique Class IDs in Frame:", unique_classes)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# Define image and label paths (update accordingly)
image_path = "/kaggle/input/tracking/tracking/tracking/train/05/img1/000001.jpg"  # Update this with your actual image path
label_path = "/kaggle/input/gtof05/gt.txt"  # Update this with your actual label path

# Extract frame number from image filename
image_filename = os.path.basename(image_path)
frame_of_interest = int(image_filename.split('.')[0])  # Convert to integer

# Load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (since OpenCV loads in BGR format)
height, width, _ = image.shape

# Read MOT label file and draw bounding boxes
unique_classes = set()

with open(label_path, "r") as file:
    lines = file.readlines()
    for line in lines:
        data = line.strip().split(',')  # Split by comma since MOT format is CSV-like
        frame_num = int(data[0])
        
        if frame_num == frame_of_interest:  # Process only the required frame
            track_id = int(data[1])
            x1 = int(float(data[2]))  # X (top-left corner)
            y1 = int(float(data[3]))  # Y (top-left corner)
            w = int(float(data[4]))   # Width
            h = int(float(data[5]))   # Height
            confidence = float(data[6])  # Confidence score
            class_id = int(data[7])  # Class ID from the file
            visibility = float(data[8])  # Visibility score
            
            unique_classes.add(class_id)
            
            if class_id == 1:  # Filter to show only class 6
                x2 = x1 + w
                y2 = y1 + h
                
                # Draw bounding box
                color = (255, 0, 0)  # Red color for bounding box
                thickness = 2
                cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
                
                # Put class label
                label_text = f"Class: {class_id} (ID: {track_id})"
                cv2.putText(image, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

# Print unique class IDs
print("Unique Class IDs in Frame:", unique_classes)


In [ ]:
# Script to extract unique class IDs from the entire GT file

import os

# Define GT file path
label_path = "/kaggle/input/gtof05/gt.txt"  # Update this with your actual label path

# Read MOT label file and extract unique class IDs
unique_classes = set()

with open(label_path, "r") as file:
    for line in file:
        data = line.strip().split(',')  # Split by comma since MOT format is CSV-like
        class_id = int(data[7])  # Extract class ID
        unique_classes.add(class_id)

# Print all unique class IDs
print("Unique Class IDs in the GT file:", unique_classes)


In [ ]:
import pandas as pd

file_path = "path/to/your/mot_annotations.txt"

# Load the MOT annotation file (adjust separator if needed)
df = pd.read_csv(file_path, sep=" ", header=None)

# The class column is the second-to-last (-2)
class_counts = df.iloc[:, -2].value_counts()

print("Class distribution in the dataset:\n", class_counts)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Define paths (update accordingly)
image_path = "/kaggle/input/tracking/tracking/tracking/train/03/img1/000063.jpg"  # Update this
label_path = "/kaggle/input/gtof03/000063.txt"  # Update this

# Load the image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
height, width, _ = image.shape

# Read YOLO label file
with open(label_path, "r") as file:
    lines = file.readlines()

# Loop through each bounding box
for line in lines:
    data = line.strip().split()
    class_id = int(data[0])  # Class ID
    x_center = float(data[1]) * width
    y_center = float(data[2]) * height
    bbox_width = float(data[3]) * width
    bbox_height = float(data[4]) * height

    # Convert to (x1, y1, x2, y2) format
    x1 = int(x_center - bbox_width / 2)
    y1 = int(y_center - bbox_height / 2)
    x2 = int(x_center + bbox_width / 2)
    y2 = int(y_center + bbox_height / 2)

    # Draw bounding box
    color = (255, 0, 0)  # Red color
    thickness = 2
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

    # Display class label
    label_text = f"Class: {class_id}"
    cv2.putText(image, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display the image with bounding boxes
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.title("YOLOv8 Bounding Boxes")
plt.show()


## Evaluation

In [ ]:
!pip install -U pip setuptools wheel
!pip install git+https://github.com/ifzhang/ByteTrack.git


In [ ]:
import numpy as np
np.float = np.float32  # Temporary fix for deprecated alias


In [ ]:
pip install deep-sort-realtime ultralytics


## Tracking

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/modeltrackingresults.txt"
visualization_output = "/kaggle/working/modeltracking_output"

# Create output folder
os.makedirs(visualization_output, exist_ok=True)

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with persistent IDs
    results = model2.track(img_resized, persist=True)[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class (1 & 7 mapped to 1)
        cls = int(track.cls[0].item())
        if cls == 0:
            # Convert to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf,4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save visualization
    cv2.imwrite(os.path.join(visualization_output, img_name), img)

# Sort tracking results by track_id first, then by frame number
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")
print(f"Visualized results saved in {visualization_output}")


In [ ]:
import shutil

# Define folder paths
visualization_output = "/kaggle/working/modeltracking_output"
zip_path = "/kaggle/working/visualized_modeltrack_output.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', visualization_output)

print(f"Zipped visualization folder saved at: {zip_path}")


## Bytetrack

In [3]:
!pip install ultralytics lap cython motmetrics onemetric


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
!pip install -e ByteTrack

In [ ]:
!pip install loguru

In [ ]:
!pip install cython_bbox

In [ ]:
import sys
sys.path.append("/kaggle/working/ByteTrack")  # Ensure ByteTrack is in path

from yolox.tracker.byte_tracker import BYTETracker  # Correct import



In [ ]:
!pip install yolox.tracker.tracking_utils

In [ ]:
!pip install -U yolox


In [ ]:
!pip install pyyaml  # (Skip this step if PyYAML is already installed)


In [ ]:
!pip install filterpy

In [ ]:
import yaml

# Define the path to your bytetrack.yaml file
file_path = "/kaggle/working/bytetrack.yaml"

# Load the existing YAML file
with open(file_path, "r") as file:
    bytetrack_config = yaml.safe_load(file)
# tracking_0.6_0.1_0.3_0.85_30_0.2_True.txt

# "tracker_type": "bytetrack",
#         "track_high_thresh": params[0],
#         "track_low_thresh": params[1],
#         "iou_thresh": params[2],
#         "match_thresh": params[3],
#         "track_buffer": params[4],
#         "new_track_thresh": params[5],
#         "fuse_score": params[6],


# Modify the values with optimized settings
bytetrack_config["tracker_type"] = "bytetrack"
bytetrack_config["track_high_thresh"] = 0.6  # Higher confidence filtering for stable tracks ##effect
bytetrack_config["track_low_thresh"] = 0.05  # Keep lower-confidence detections to maintain recall ##no
bytetrack_config["new_track_thresh"] = 0.2  # Prevent frequent new ID assignments #no
bytetrack_config["iou_thresh"] = 0.7  # Higher IoU helps in tracking objects with slight movement #no
bytetrack_config["match_thresh"] = 0.83  # Reduce ID switches by making matching stricter #bigeffect
bytetrack_config["track_buffer"] = 30  # Increase buffer to maintain identity over more frames #small 
bytetrack_config["fuse_score"] = True  # Enable score fusion for better tracking #true better
bytetrack_config["frame_rate"] = 25  # Ensure tracking is adjusted for 25 FPS videos
bytetrack_config["det_thresh"] = 0.5  # Detection confidence threshold #no effect
bytetrack_config["lost_track_thresh"] = 10  # Use confidence fusion for better tracking #no effect
# Save the updated YAML file
with open(file_path, "w") as file:
    yaml.dump(bytetrack_config, file, default_flow_style=False)

print("bytetrack.yaml has been updated successfully with optimized settings!")


In [ ]:
!pip install -q strongsort


In [ ]:
!git clone https://github.com/NirAharon/BoT-SORT.git
%cd BoT-SORT
!pip install -r requirements.txt

In [ ]:
import sys
sys.path.append("/kaggle/working/BoT-SORT")  # Adjust this path if needed


In [ ]:
!pip install fastreid

In [ ]:
import collections
if not hasattr(collections, 'Mapping'):
    import collections.abc
    collections.Mapping = collections.abc.Mapping
file_path = "/kaggle/working/BoT-SORT/fast_reid/fastreid/data/build.py"

# Read the file and modify the import statement
with open(file_path, "r") as file:
    lines = file.readlines()

new_lines = []
for line in lines:
    if "from collections import Mapping" in line:
        new_lines.append("from collections.abc import Mapping\n")
    else:
        new_lines.append(line)

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(new_lines)

print("✅ Fix applied successfully!")


In [ ]:
file_path = "/kaggle/working/BoT-SORT/fast_reid/fastreid/evaluation/testing.py"

# Read the file and modify the import statement
with open(file_path, "r") as file:
    lines = file.readlines()

new_lines = []
for line in lines:
    if "from collections.abc import Mapping, OrderedDict" in line:
        new_lines.append("from collections.abc import Mapping\nfrom collections import OrderedDict\n")
    else:
        new_lines.append(line)

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(new_lines)

print("✅ Fix applied successfully!")


In [ ]:
!sed -i 's/from collections import Mapping, OrderedDict/from collections.abc import Mapping, OrderedDict/' /kaggle/working/BoT-SORT/fast_reid/fastreid/evaluation/testing.py
!grep "Mapping" /kaggle/working/BoT-SORT/fast_reid/fastreid/evaluation/testing.py
file_path = "/kaggle/working/BoT-SORT/fast_reid/fastreid/data/build.py"

# Read the file and modify the import statement
with open(file_path, "r") as file:
    lines = file.readlines()

new_lines = []
for line in lines:
    if "from torch._six import string_classes" in line:
        new_lines.append("string_classes = (str,)\n")
    else:
        new_lines.append(line)

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(new_lines)

print("✅ Fix applied successfully!")
file_path = "/kaggle/working/BoT-SORT/fast_reid/fastreid/evaluation/testing.py"

# Read the file
with open(file_path, "r") as file:
    lines = file.readlines()

# Modify only the incorrect import
for i in range(len(lines)):
    if "from collections.abc import Mapping, OrderedDict" in lines[i]:
        lines[i] = "from collections import OrderedDict\nfrom collections.abc import Mapping\n"

# Write back the corrected file
with open(file_path, "w") as file:
    file.writelines(lines)

print(f"File '{file_path}' has been updated successfully!")


In [ ]:
!pip uninstall fastreid -y
!pip install git+https://github.com/JDAI-CV/fast-reid.git


In [ ]:
file_path = "/kaggle/working/BoT-SORT/fast_reid/fastreid/data/build.py"

# Read the file and modify the import statement
with open(file_path, "r") as file:
    lines = file.readlines()

new_lines = []
for line in lines:
    if "from torch._six import string_classes" in line:
        new_lines.append("string_classes = (str,)\n")
    else:
        new_lines.append(line)

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(new_lines)

print("✅ Fix applied successfully!")


In [ ]:
from fastreid.config import get_cfg
from fastreid.engine import DefaultTrainer

# Load FastReID model
cfg = get_cfg()
cfg.merge_from_file("/kaggle/working/BoT-SORT/fast_reid/configs/MOT20/sbs_S50.yml")  # Config file
cfg.MODEL.WEIGHTS = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"

reid_model = DefaultTrainer.build_model(cfg)
reid_model.eval()



In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print(torch.__version__)
print(dir(torch._subclasses))

In [ ]:
!pip install fastreid --upgrade

In [ ]:
!pip install torch==1.12.1 torchvision==0.13.1


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
from fastreid.config import get_cfg
from fastreid.engine import DefaultTrainer
from torchvision import transforms

# Load YOLOv8 fine-tuned model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Load ReID model
cfg = get_cfg()
cfg.merge_from_file("/kaggle/working/BoT-SORT/fast_reid/configs/MOT20/sbs_S50.yml")  
cfg.MODEL.WEIGHTS = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"
reid_model = DefaultTrainer.build_model(cfg)
reid_model.eval().cuda()

# Transformation for ReID model
reid_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 128)),  # Expected input size for ReID
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/bytetracking_ftlast_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Dictionary to store ReID embeddings per track_id
track_embeddings = {}

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with ByteTrack
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/bytetrack.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        # Process only persons (class 0)
        if cls == 0:
            # Convert coordinates to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w, h = int(w), int(h)

            # Extract person crop
            person_crop = img[max(0, y): min(original_height, y + h), max(0, x): min(original_width, x + w)]
            
            if person_crop.size > 0:
                # Extract ReID embedding
                person_tensor = reid_transform(person_crop).unsqueeze(0).cuda()
                with torch.no_grad():
                    embedding = reid_model(person_tensor).cpu().numpy()

                # Store embeddings for track_id
                if track_id in track_embeddings:
                    track_embeddings[track_id].append(embedding)
                else:
                    track_embeddings[track_id] = [embedding]

                # Update track_id based on ReID similarity
                best_match_id = track_id
                best_match_score = float('inf')

                for existing_id, embeddings in track_embeddings.items():
                    avg_embedding = np.mean(embeddings, axis=0)
                    dist = np.linalg.norm(embedding - avg_embedding)

                    if dist < best_match_score:
                        best_match_score = dist
                        best_match_id = existing_id

                track_id = best_match_id  # Assign best match ID

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort tracking results by track_id first, then by frame number
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/bytetracking_ftlast_results.txt"
#visualization_output = "/kaggle/working/bytetracking_ft2_output"

# Create output folder
#os.makedirs(visualization_output, exist_ok=True)

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with StrongSORT and persistent IDs
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/bytetrack.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class (1 & 7 mapped to 1)
        cls = int(track.cls[0].item())
        if cls == 0:
            # Convert to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save visualization
    #cv2.imwrite(os.path.join(visualization_output, img_name), img)

# Sort tracking results by track_id first, then by frame number
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")
#print(f"Visualized results saved in {visualization_output}")


In [ ]:
# Create folder for extracted Re-ID dataset
reid_dataset_path = "/kaggle/working/reid_dataset"
os.makedirs(reid_dataset_path, exist_ok=True)

for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  
    img_path = os.path.join(test_images_path, img_name)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue

    img_resized = cv2.resize(img, (target_width, target_height))
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/bytetrack.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1  
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        if cls == 0:
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Save cropped images for Re-ID training
            person_crop = img[y:y + h, x:x + w]
            if person_crop.shape[0] > 10 and person_crop.shape[1] > 10:  # Avoid small false detections
                crop_filename = f"{reid_dataset_path}/{track_id}_{frame_number}.jpg"
                cv2.imwrite(crop_filename, person_crop)


In [ ]:
# Install dependencies
!pip install torch torchvision opencv-python
!git clone https://github.com/JDAI-CV/fast-reid.git
!cd fast-reid
!pip install -r requirements.txt


In [ ]:
# Change dataset path
DATASET_PATH = "/kaggle/working/reid_dataset"
!export PYTHONPATH="/kaggle/working/fast-reid:$PYTHONPATH"
!python /kaggle/working/fast-reid/tools/train_net.py \
  --config-file /kaggle/working/fast-reid/configs/Market1501/bagtricks_R50.yml \
  --num-gpus 1 \
  OUTPUT_DIR logs/reid_mot20



In [ ]:
import shutil

# Define folder paths
visualization_output = "/kaggle/working/bytebestmodel_output"
zip_path = "/kaggle/working/visualizedbytebest_output.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', visualization_output)

print(f"Zipped visualization folder saved at: {zip_path}")


## DEEP SORT

In [ ]:
!pip install deep_sort_realtime ultralytics


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Initialize DeepSORT with tuned parameters for MOT20
deep_sort = DeepSort(
    max_age=70,  # Increased for MOT20 longer occlusions
    n_init=3,  # Lower for faster ID assignment
    nms_max_overlap=0.7,  # Adjust overlap threshold
    max_cosine_distance=0.2,  # Lower distance for better ID matching
    nn_budget=100  # Larger budget for MOT20 complexity
)

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/deepsort_mot20_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLO inference
    results = model2.predict(img_resized, conf=0.5, iou=0.5)[0]  # Adjust confidence & IOU thresholds
    detections = []
    track_confidences = {}  # Store confidences separately

    for det in results.boxes:
        x_center, y_center, w, h = det.xywh[0].cpu().numpy()
        conf = det.conf[0].item()
        cls = int(det.cls[0].item())

        # Keep only person class
        if cls == 0:
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)

            bbox = [x, y, w, h]
            detections.append((bbox, conf, cls))
            track_confidences[str(bbox)] = conf  # Store confidence with bbox as key

    # Run DeepSORT tracking
    tracks = deep_sort.update_tracks(detections, frame=img)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltrb()  # Get left-top-right-bottom format
        x1, y1, x2, y2 = map(int, bbox)
        w, h = x2 - x1, y2 - y1

        # Retrieve confidence from stored values
        conf = track_confidences.get(str([x1, y1, w, h]), 0.0)

        # Store tracking results in MOT format
        tracking_results.append([frame_number, track_id, x1, y1, w, h, round(conf, 4), 1, 1])

        # Draw bounding box and ID
        cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        cv2.putText(img, f"ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):  # Sort by ID, then frame
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")


In [ ]:
import shutil

# Define folder paths
visualization_output = "/kaggle/working/visualized_deepsort_best_output"
zip_path = "/kaggle/working/visualized_deep_output.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', visualization_output)

print(f"Zipped visualization folder saved at: {zip_path}")


## Strong SORT

In [ ]:
%cd /kaggle/working

In [27]:
!pip install ultralytics lap thop torchvision
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/bytetrack.yaml
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/strongsort.yaml


--2025-03-22 18:11:34--  https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/bytetrack.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 886 [text/plain]
Saving to: ‘bytetrack.yaml’

bytetrack.yaml      100%[===================>]     886  --.-KB/s    in 0s      

2025-03-22 18:11:35 (35.2 MB/s) - ‘bytetrack.yaml’ saved [886/886]

--2025-03-22 18:11:35--  https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/strongsort.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404

In [ ]:
!echo "tracker_type: strongsort" > strongsort.yaml


In [ ]:
strongsort_yaml = """
tracker_type: strongsort
model_weights: osnet_x0_25_market1501.pt
max_age: 30
min_hits: 3
iou_threshold: 0.3
"""

with open("strongsort.yaml", "w") as f:
    f.write(strongsort_yaml)


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/sstrackingresults.txt"
visualization_output = "/kaggle/working/sstracking_output"

# Create output folder
os.makedirs(visualization_output, exist_ok=True)

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with StrongSORT and persistent IDs
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/strongsort.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class (1 & 7 mapped to 1)
        cls = int(track.cls[0].item())
        if cls == 0:
            # Convert to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save visualization
    cv2.imwrite(os.path.join(visualization_output, img_name), img)

# Sort tracking results by track_id first, then by frame number
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")
print(f"Visualized results saved in {visualization_output}")


## BOT-SORT

In [ ]:
botsort_config = """
tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.03
new_track_thresh: 0.2
iou_thresh: 0.35
match_thresh: 0.9
track_buffer: 90
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 120
proximity_thresh: 0.5
appearance_thresh: 0.28
with_reid: True
reid_model: /kaggle/input/reidsbs/mot20_sbs_S50.pth
gmc_method: sparseOptFlow
"""

with open("/kaggle/working/botsort.yaml", "w") as file:
    file.write(botsort_config)

print("BoT-SORT YAML saved successfully!")


In [ ]:
from botsort import BoTSORT

# Load BoT-SORT with ReID enabled
tracker = BoTSORT("/kaggle/working/botsort.yaml")

# Print ReID model status
if tracker.model is None:
    print("❌ ReID model NOT loaded")
else:
    print("✅ ReID model successfully loaded")


In [ ]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/botsort_results.txt"

# Image resize settings
ORIG_W, ORIG_H = 1920, 1080
TARGET_W, TARGET_H = 704, 704

# Store tracking results
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_num = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Skipping {img_path} (load error)")
        continue

    # Resize image for YOLO
    img_resized = cv2.resize(img, (TARGET_W, TARGET_H))

    # Run BoT-SORT tracking
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/botsort.yaml", verbose=True)[0]

    for track in results.boxes:
        x_c, y_c, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        if cls == 0 and conf > 0.25:  # Keep only 'person' class with high confidence
            # Scale back to original image size
            x_c *= (ORIG_W / TARGET_W)
            y_c *= (ORIG_H / TARGET_H)
            w *= (ORIG_W / TARGET_W)
            h *= (ORIG_H / TARGET_H)

            # Convert to top-left format
            x, y = int(x_c - w / 2), int(y_c - h / 2)
            w, h = int(w), int(h)

            # Save results in MOT format
            tracking_results.append([frame_num, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort results: Track ID first, then frame number
track_id_data = {}

for row in tracking_results:
    f_num, t_id = row[0], row[1]
    if t_id not in track_id_data:
        track_id_data[t_id] = []
    track_id_data[t_id].append(row)

# Save in MOT format
with open(output_mot_path, "w") as f:
    for t_id in sorted(track_id_data.keys()):  
        for row in sorted(track_id_data[t_id], key=lambda x: x[0]):  
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")


In [ ]:
import torch

reid_model_path = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    reid_model = torch.load(reid_model_path, map_location=device)
    print("✅ ReID model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load ReID model: {e}")


## MOTA EVALUATION

In [5]:
!pip install motmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
!pip uninstall -y motmetrics


In [ ]:
!pip install --upgrade git+https://github.com/cheind/py-motmetrics.git


In [ ]:
import numpy as np
import motmetrics as mm
import re

# Locate mot.py file inside the motmetrics package
mot_file_path = mm.__file__.replace('__init__.py', 'mot.py')

# Read and fix the deprecated np.bool usage
with open(mot_file_path, 'r') as file:
    content = file.read()

# Replace np.bool with np.bool_
fixed_content = re.sub(r'np.bool', 'np.bool_', content)

# Write the fixed version back
with open(mot_file_path, 'w') as file:
    file.write(fixed_content)

print("Patched motmetrics successfully!")


In [ ]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/bytetracking_ftlast_results.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


## Tracking Submission

In [4]:
import pandas as pd
import ast

# Load MOT tracking results
def load_mot_results(mot_file):
    tracking_data = {}
    with open(mot_file, 'r') as f:
        for line in f:
            frame_id, track_id, x, y, w, h, conf, *_ = map(float, line.strip().split(','))
            if int(frame_id) not in tracking_data:
                tracking_data[int(frame_id)] = []
            tracking_data[int(frame_id)].append({
                "tracked_id": track_id, "x": int(x), "y": int(y),
                "w": int(w), "h": int(h), "confidence": conf
            })
    return tracking_data

# Load submission template
def load_submission_template(submission_file):
    df = pd.read_csv(submission_file)
    return df

# Process and fill submission file
def fill_submission_file(mot_file, submission_file, output_file):
    tracking_data = load_mot_results(mot_file)
    df = load_submission_template(submission_file)
    
    for i in range(len(df)):
        if df.loc[i, "objective"] == "tracking":
            frame_id = int(df.loc[i, "frame"])  # Get frame from submission template
            if frame_id in tracking_data:
                df.loc[i, "objects"] = str(tracking_data[frame_id])  # Fill tracking results
        elif df.loc[i, "objective"] == "face_reid":
            df.loc[i, "frame"] = -1  # Ensure face_reid has frame -1
            df.loc[i, "objects"] = str(ast.literal_eval(df.loc[i, "objects"]))  # Keep original

    # Save final submission file
    df.to_csv(output_file, index=False)
    print(f"Submission file saved as: {output_file}")

# Example usage
mot_file = "/kaggle/working/modeltrackingresults.txt"
submission_file = "/kaggle/input/fawrysub1/sub1.csv"
output_file = "/kaggle/working/50.csv"

fill_submission_file(mot_file, submission_file, output_file)


Submission file saved as: /kaggle/working/50.csv


## Grid Search

In [ ]:
import yaml
import itertools
import numpy as np
import os
import cv2
from ultralytics import YOLO

# Define paths
file_path = "/kaggle/working/bytetrack.yaml"
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_dir = "/kaggle/working/mot_results"
os.makedirs(output_dir, exist_ok=True)

# Optimized parameter ranges for tuning
param_grid = {
    "track_high_thresh": [0.55, 0.6, 0.65],
    "track_low_thresh": [0.05, 0.1, 0.15],
    "iou_thresh": [0.25, 0.3, 0.35],
    "match_thresh": [0.8, 0.85, 0.9],
    "track_buffer": [20, 30, 40],
    "new_track_thresh": [0.15, 0.2, 0.25],
    "fuse_score": [True]
}

# Generate all parameter combinations
param_combinations = list(itertools.product(*param_grid.values()))

# Model setup
model = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Image resize parameters
original_width = 1920
original_height = 1080
target_width = 704
target_height = 704

for params in param_combinations:
    # Update tracker configuration
    bytetrack_config = {
        "tracker_type": "bytetrack",
        "track_high_thresh": params[0],
        "track_low_thresh": params[1],
        "iou_thresh": params[2],
        "match_thresh": params[3],
        "track_buffer": params[4],
        "new_track_thresh": params[5],
        "fuse_score": params[6],
    }
    
    with open(file_path, "w") as file:
        yaml.dump(bytetrack_config, file, default_flow_style=False)
    
    # Generate unique filename
    param_str = "_".join(map(str, params))
    mot_output_path = os.path.join(output_dir, f"tracking_{param_str}.txt")
    tracking_results = []
    
    # Process test images
    for img_name in sorted(os.listdir(test_images_path)):
        frame_number = int(img_name.split(".")[0])
        img_path = os.path.join(test_images_path, img_name)

        img = cv2.imread(img_path)
        if img is None:
            continue

        img_resized = cv2.resize(img, (target_width, target_height))
        results = model.track(img_resized, persist=True, tracker=file_path)[0]

        for track in results.boxes:
            x_center, y_center, w, h = track.xywh[0].cpu().numpy()
            track_id = int(track.id[0].item()) if track.id is not None else -1
            conf = track.conf[0].item()
            cls = int(track.cls[0].item())
            if cls == 0:
                x_center *= (original_width / target_width)
                y_center *= (original_height / target_height)
                w *= (original_width / target_width)
                h *= (original_height / target_height)
                x = int(x_center - w / 2)
                y = int(y_center - h / 2)
                w = int(w)
                h = int(h)
                tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])
    
    # Save results
    with open(mot_output_path, "w") as f:
        for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):
            f.write(",".join(map(str, row)) + "\n")
    
    print(f"Saved results to {mot_output_path}")


In [ ]:
import os
import motmetrics as mm
import numpy as np

def load_tracks(file_path):
    """Load tracking data from a file."""
    return np.loadtxt(file_path, delimiter=',')

# Define paths
gt_file = "/kaggle/input/filteredgt/filtered_gt.txt"  # Ground truth file path
results_dir = "/kaggle/input/gttrials"  # Directory with tracking result files

gt_data = load_tracks(gt_file)
best_mota = -np.inf
best_file = None

# Iterate over each tracking result file
for file_name in os.listdir(results_dir):
    if file_name.endswith(".txt"):
        result_path = os.path.join(results_dir, file_name)
        pred_data = load_tracks(result_path)
        
        # Create an accumulator to store results
        acc = mm.MOTAccumulator(auto_id=True)

        # Iterate through each frame
        for frame in np.unique(gt_data[:, 0]):
            # Extract ground truth and predictions for the current frame
            gt_objects = gt_data[gt_data[:, 0] == frame]
            pred_objects = pred_data[pred_data[:, 0] == frame]

            # Extract bounding boxes and IDs
            gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
            gt_ids = gt_objects[:, 1]

            pred_boxes = pred_objects[:, 2:6]
            pred_ids = pred_objects[:, 1]

            # Compute pairwise distances (IoU-based)
            dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)
            
            # Update the accumulator
            acc.update(gt_ids, pred_ids, dist_matrix)

        # Compute metrics
        mh = mm.metrics.create()
        summary = mh.compute(acc, metrics=['mota'], name=file_name)
        mota_score = summary.loc[file_name]['mota']
        print(f"{file_name}: MOTA = {mota_score}")
        
        # Check for the best MOTA score
        if mota_score > best_mota:
            best_mota = mota_score
            best_file = file_name

print(f"\nBest Tracking Result: {best_file} with MOTA = {best_mota}")


## REID

In [ ]:
ls /kaggle/working/models/vit_kprpe/RPE/rpe_ops/


In [ ]:
!pip install torch torchvision torchaudio
!pip install setuptools wheel ninja


In [ ]:
!pip install .


In [ ]:
!ls build/
!ls rpe_index.egg-info/


In [ ]:
!ls -l /kaggle/working/models/vit_kprpe/


In [ ]:
!python setup.py build_ext --inplace


In [ ]:
ls -l /kaggle/working/models/vit_kprpe/RPE/


In [ ]:
pip install -e /kaggle/working/models/vit_kprpe/RPE/


In [ ]:
ls -l /kaggle/working/models/vit_kprpe/RPE/


In [ ]:
ls -l /kaggle/working/models/vit_kprpe/RPE/rpe_ops/


In [ ]:
cd /kaggle/working/models/vit_kprpe/RPE/rpe_ops/


In [ ]:
!python /kaggle/working/models/vit_kprpe/RPE/rpe_ops/setup.py build_ext --inplace


In [ ]:
import sys
sys.path.append("/kaggle/working/models/vit_kprpe/RPE/")
import rpe_ops


## BOT OR OC

In [6]:
# # BoT-SORT Configuration
botsort_config = """
tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.1
new_track_thresh: 0.2
iou_thresh: 0.3
match_thresh: 0.85
track_buffer: 30
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 50
proximity_thresh: 0.5
appearance_thresh: 0.25
with_reid: False
gmc_method: sparseOptFlow
"""

# OC-SORT Configuration
ocsort_config = """
tracker_type: ocsort
track_high_thresh: 0.6
track_low_thresh: 0.1
new_track_thresh: 0.2
iou_thresh: 0.3
match_thresh: 0.85
track_buffer: 30
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 50
"""

# Save BoT-SORT configuration to botsort.yaml
with open("/kaggle/working/botsort.yaml", "w") as file:
    file.write(botsort_config)

# Save OC-SORT configuration to ocsort.yaml
with open("/kaggle/working/ocsort.yaml", "w") as file:
    file.write(ocsort_config)

print("YAML files created successfully in /kaggle/working!")

YAML files created successfully in /kaggle/working!


In [ ]:
!pip install -U ultralytics

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path_botsort = "/kaggle/working/botsort_results.txt"
output_mot_path_ocsort = "/kaggle/working/ocsort_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Function to run tracking
def run_tracking(tracker_config, output_mot_path):
    tracking_results = []

    # Process test frames
    for img_name in sorted(os.listdir(test_images_path)):
        frame_number = int(img_name.split(".")[0])  # Extract frame number
        img_path = os.path.join(test_images_path, img_name)

        # Load and resize image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading {img_path}, skipping.")
            continue
        
        img_resized = cv2.resize(img, (target_width, target_height))

        # Run YOLOv8 tracking with the specified tracker
        results = model2.track(img_resized, persist=True, tracker="/kaggle/working/botsort.yaml")[0]  

        for track in results.boxes:
            x_center, y_center, w, h = track.xywh[0].cpu().numpy()
            track_id = int(track.id[0].item()) if track.id is not None else -1  # Ensure track_id persists
            conf = track.conf[0].item()

            # Keep only person class (1 & 7 mapped to 1)
            cls = int(track.cls[0].item())
            if cls == 0:
                # Convert to original image size
                x_center *= (original_width / target_width)
                y_center *= (original_height / target_height)
                w *= (original_width / target_width)
                h *= (original_height / target_height)

                # Convert to top-left format
                x = int(x_center - w / 2)
                y = int(y_center - h / 2)
                w = int(w)
                h = int(h)

                # Save tracking result in MOT format
                tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

    # Sort tracking results by track_id first, then by frame number
    track_id_data = {}

    for row in tracking_results:
        frame_number, track_id = row[0], row[1]
        if track_id not in track_id_data:
            track_id_data[track_id] = []
        track_id_data[track_id].append(row)

    # Save results in MOT format
    with open(output_mot_path, "w") as f:
        for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
            for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
                f.write(",".join(map(str, row)) + "\n")

    print(f"Tracking results saved to {output_mot_path}")

# Run BoT-SORT
run_tracking("/kaggle/working/botsort.yaml", output_mot_path_botsort)

# Run OC-SORT
run_tracking("/kaggle/working/ocsort.yaml", output_mot_path_ocsort)

## Bot with REID

In [ ]:
# BoT-SORT Configuration with ReID
botsort_config = """
tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.05
new_track_thresh: 0.15
iou_thresh: 0.4
match_thresh: 0.9
track_buffer: 60
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 100
proximity_thresh: 0.4
appearance_thresh: 0.2
with_reid: True  # Enable ReID
reid_model: /kaggle/input/reidsbs/mot20_sbs_S50.pth  # Path to the ReID model
gmc_method: sparseOptFlow
"""

# Save BoT-SORT configuration to botsort.yaml
with open("/kaggle/working/botsort.yaml", "w") as file:
    file.write(botsort_config)

print("botsort.yaml updated successfully in /kaggle/working!")

In [ ]:
!git clone https://github.com/NirAharon/BOT-SORT.git
%cd BOT-SORT

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install .

In [ ]:
pip list | grep bot-sort

In [ ]:
%cd /kaggle/working/BOT-SORT
!pip install .

In [ ]:
!pip install cython_bbox

In [ ]:
!find /kaggle/working/BOT-SORT/fast_reid -type f -name "*.py" -exec sed -i 's/from collections import Mapping/from collections.abc import Mapping/g' {} +


In [ ]:
!pip install faiss-gpu


In [ ]:
import os

# Path to the matching.py file
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    lines = file.readlines()

# Replace np.float with np.float64
with open(file_path, "w") as file:
    for line in lines:
        file.write(line.replace("np.float", "np.float64"))

print(f"Updated {file_path} to replace np.float with np.float64.")

In [ ]:
!pip install /path/to/numpy-1.19.3-cpXX-cpXX-win_amd64.whl

!pip install numpy==1.19.3



In [ ]:
import os
import cv2
import torch
import numpy as np
import argparse
from ultralytics import YOLO
from bot_sort import BoTSORT  # Import BoT-SORT

# Initialize the YOLO model
def load_yolo_model(model_path):
    return YOLO(model_path)

# Initialize BoT-SORT tracker
def initialize_tracker(args):
    return BoTSORT(args, frame_rate=25)

# Resize image while maintaining the aspect ratio
def resize_image(image, target_width, target_height):
    return cv2.resize(image, (target_width, target_height))

# Process YOLOv8 detections and convert to bounding box format
def process_yolo_detections(results, original_width, original_height, target_width, target_height):
    detections = []
    for box in results.boxes:
        x_center, y_center, w, h = box.xywh[0].cpu().numpy()
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())

        if cls == 0:  # Only process 'person' class
            # Rescale bounding box to original size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            detections.append([x, y, x + w, y + h, conf, cls])  # Ensure correct shape
    return np.array(detections, dtype=np.float64) if len(detections) > 0 else np.empty((0, 6), dtype=np.float64)

# Save tracking results to a file in MOT format
def save_tracking_results(tracking_results, output_path):
    track_id_data = {}
    for row in tracking_results:
        frame_number, track_id = row[0], row[1]
        if track_id not in track_id_data:
            track_id_data[track_id] = []
        track_id_data[track_id].append(row)

    with open(output_path, "w") as f:
        for track_id in sorted(track_id_data.keys()):
            for row in sorted(track_id_data[track_id], key=lambda x: x[0]):
                f.write(",".join(map(str, row)) + "\n")

# Main tracking loop
def main(model_path, reid_weights, test_images_path, output_mot_path, target_width=704, target_height=704):
    # Load models and initialize tracker
    yolo_model = load_yolo_model(model_path)
    args = argparse.Namespace(
        reid_weights=reid_weights,
        track_high_thresh=0.5,         
        track_low_thresh=0.1, 
        name="BoTSORT", 
        ablation=False, 
        new_track_thresh=0.7,          
        track_buffer=30,               
        match_thresh=0.8,              
        aspect_ratio_thresh=1.6,       
        min_box_area=10,               
        mot20=False,                   
        proximity_thresh=0.5,          
        appearance_thresh=0.25,        
        with_reid=True, 
        fast_reid_config="/kaggle/working/BOT-SORT/fast_reid/configs/MOT20/sbs_S50.yml",
        fast_reid_weights="/kaggle/input/reidsbs/mot20_sbs_S50.pth",
        device="cuda",  
        cmc_method="sparseOptFlow"
    )
    tracker = initialize_tracker(args)

    # Tracking results storage
    tracking_results = []

    # Process test frames
    for img_name in sorted(os.listdir(test_images_path)):
        frame_number = int(img_name.split(".")[0])  # Extract frame number
        img_path = os.path.join(test_images_path, img_name)

        # Load and resize image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading {img_path}, skipping.")
            continue
        
        img_resized = resize_image(img, target_width, target_height)

        # Run YOLOv8 detection
        results = yolo_model(img_resized)[0]
        detections = process_yolo_detections(results, 1920, 1080, target_width, target_height)

        # Run BoT-SORT tracking
        tracks = tracker.update(detections, img)

        for track in tracks:
            if len(track) < 6:  # Ensure track has correct columns
                continue

            track_id = int(track[4])  # Get track ID
            x, y, x2, y2 = map(int, track[:4])  # Convert bounding box coordinates
            w, h = x2 - x, y2 - y  # Convert to width-height format
            conf = round(float(track[5]), 4)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, conf, 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save tracking results to MOT format
    save_tracking_results(tracking_results, output_mot_path)
    print(f"Tracking results saved to {output_mot_path}")

# Define paths
model_path = "/kaggle/input/model-trial/pytorch/default/3/best (1).pt"
reid_weights = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/botsort_results.txt"

# Run the main function
main(model_path, reid_weights, test_images_path, output_mot_path)


In [ ]:
# Find all occurrences of dtype=float in the BoT-SORT directory
!grep -r "dtype=float" /kaggle/working/BOT-SORT/

In [ ]:
import os
import re

# Regex patterns to find and replace
patterns = [
    (r'dtype=float\)', r'dtype=np.float64)'),
    (r'dtype=float,', r'dtype=np.float64,'),
    (r'dtype=float32\)', r'dtype=np.float32)'),
    (r'dtype=float32,', r'dtype=np.float32,'),
    (r'dtype=float64\)', r'dtype=np.float64)'),
    (r'dtype=float64,', r'dtype=np.float64,'),
    (r'dtype=float_\)', r'dtype=np.float64)'),  # Assuming float_ was meant to be float64
]

# Get all Python files in the BOT-SORT directory
def fix_numpy_types(directory):
    fixed_files = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                # Read the file content
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                
                # Apply all patterns
                original_content = content
                for pattern, replacement in patterns:
                    content = re.sub(pattern, replacement, content)
                
                # If any changes were made, write the file back
                if content != original_content:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(content)
                    fixed_files += 1
                    print(f"Fixed {file_path}")
    
    return fixed_files

# Run the fix on BOT-SORT directory
bot_sort_dir = "/kaggle/working/BOT-SORT"
total_fixed = fix_numpy_types(bot_sort_dir)
print(f"\nTotal files fixed: {total_fixed}")

# Fix the specific matching.py file that caused the error
matching_file = "/kaggle/working/BOT-SORT/tracker/matching.py"
if os.path.exists(matching_file):
    with open(matching_file, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    
    # Replace specific instances in matching.py
    content = content.replace("dtype=float", "dtype=np.float64")
    
    with open(matching_file, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"Specifically fixed {matching_file}")

print("\nDone! Try running your tracking code again now.")

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/bot_sort.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace np.float with float
updated_content = content.replace("np.float", "float")

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
def ious(atlbrs, btlbrs):
    """
    Compute cost based on IoU
    :type atlbrs: list[tlbr] | np.ndarray
    :type atlbrs: list[tlbr] | np.ndarray

    :rtype ious np.ndarray
    """
    ious = np.zeros((len(atlbrs), len(btlbrs)), dtype=float)  # Fixed: Use float instead of np.float64
    if ious.size == 0:
        return ious

    ious = bbox_ious(
        np.ascontiguousarray(atlbrs, dtype=float),  # Fixed: Use float instead of np.float64
        np.ascontiguousarray(btlbrs, dtype=float)   # Fixed: Use float instead of np.float64
    )

    return ious

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace the ious function with the corrected version
corrected_function = """
def ious(atlbrs, btlbrs):
    \"\"\"
    Compute cost based on IoU
    :type atlbrs: list[tlbr] | np.ndarray
    :type atlbrs: list[tlbr] | np.ndarray

    :rtype ious np.ndarray
    \"\"\"
    ious = np.zeros((len(atlbrs), len(btlbrs)), dtype=float)  # Fixed: Use float instead of np.float64
    if ious.size == 0:
        return ious

    ious = bbox_ious(
        np.ascontiguousarray(atlbrs, dtype=float),  # Fixed: Use float instead of np.float64
        np.ascontiguousarray(btlbrs, dtype=float)   # Fixed: Use float instead of np.float64
    )

    return ious
"""

# Replace the old ious function with the corrected version
updated_content = content.replace(
    """
def ious(atlbrs, btlbrs):
    \"\"\"
    Compute cost based on IoU
    :type atlbrs: list[tlbr] | np.ndarray
    :type atlbrs: list[tlbr] | np.ndarray

    :rtype ious np.ndarray
    \"\"\"
    ious = np.zeros((len(atlbrs), len(btlbrs)), dtype=np.float64)
    if ious.size == 0:
        return ious

    ious = bbox_ious(
        np.ascontiguousarray(atlbrs, dtype=np.float64),
        np.ascontiguousarray(btlbrs, dtype=np.float64)
    )

    return ious
""",
    corrected_function
)

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

with open(file_path, "r") as file:
    for line in file:
        if "dtype=float" in line:
            print("Change found:", line.strip())

In [ ]:
# Navigate to the BOT-SORT directory
%cd /kaggle/working/BOT-SORT

# Clean the build directory (if it exists)
!rm -rf build

# Rebuild the project
!python setup.py install

In [ ]:
import numpy as np
print(np.__version__)

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace np.float64 with float
updated_content = content.replace("dtype=np.float64", "dtype=float")

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace float64 with np.float64
updated_content = content.replace("dtype=float64", "dtype=np.float64")

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
import os

def search_for_np_float(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    lines = f.readlines()
                    for line_num, line in enumerate(lines, 1):
                        if "np.float" in line:
                            print(f"Found 'np.float' in {file_path}, line {line_num}: {line.strip()}")

# Specify the BOT-SORT directory
bot_sort_directory = "/kaggle/working/BOT-SORT"
search_for_np_float(bot_sort_directory)

In [ ]:
import os

def replace_np_float(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    content = f.read()
                
                # Replace np.float with np.float32
                updated_content = content.replace("np.float", "np.float32")
                
                # Write the updated content back to the file
                with open(file_path, "w") as f:
                    f.write(updated_content)

# Specify the BOT-SORT directory
bot_sort_directory = "/kaggle/working/BOT-SORT"
replace_np_float(bot_sort_directory)

print("All occurrences of 'np.float' have been replaced with 'np.float32'.")

In [ ]:
import os

def fix_damaged_lines(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    content = f.read()
                
                # Fix np.float3232 -> np.float32
                content = content.replace("np.float3232", "np.float32")
                # Fix np.float3264 -> np.float64
                content = content.replace("np.float3264", "np.float64")
                
                # Write the updated content back to the file
                with open(file_path, "w") as f:
                    f.write(content)

# Specify the BOT-SORT directory
bot_sort_directory = "/kaggle/working/BOT-SORT"
fix_damaged_lines(bot_sort_directory)

print("All damaged lines have been fixed.")

In [ ]:
# Navigate to the BOT-SORT directory
%cd /kaggle/working/BOT-SORT

# Clean the build directory (if it exists)
!rm -rf build

# Rebuild the project
!python setup.py install

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace np.float with np.float64
updated_content = content.replace("np.float", "np.float64")

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

with open(file_path, "r") as file:
    for line in file:
        if "np.float64" in line:
            print("Change found:", line.strip())

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

with open(file_path, "r") as file:
    for line in file:
        if "np.float64" in line:  # Properly indented
            print("Change found:", line.strip())

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

with open(file_path, "r") as file:
    print(file.read())

In [ ]:
file_path = "/kaggle/working/BOT-SORT/tracker/matching.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace float64 with np.float64
updated_content = content.replace("dtype=float64", "dtype=np.float64")

# Write the updated content back to the file
with open(file_path, "w") as file:
    file.write(updated_content)

print("File updated successfully!")

In [ ]:
import torch

reid_model_path = "/kaggle/working/BOT-SORT/fast_reid/mot17_sbs_S50.pth"

try:
    model_data = torch.load(reid_model_path, map_location="cpu")
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")


In [ ]:
!find /kaggle/working/BOT-SORT -type f -name "*.py" -exec sed -i 's/from torch._six import string_classes/import collections\nstring_classes = (str, bytes)/g' {} +

In [ ]:
ls -lh /kaggle/working/BOT-SORT/fast_reid/mot17_sbs_S50.pth


In [ ]:
%cd /kaggle/working/BOT-SORT/fast_reid/fastreid



In [ ]:
!wget -O mot20_sbs_S50.pth https://github.com/NirAharon/BoT-SORT


In [ ]:
!rm -f /kaggle/working/BOT-SORT/fast_reid/fastreid/mot20_sbs_S50.pth


In [ ]:
!pip install lap

In [ ]:
!pip install yacs

In [ ]:
import sys
sys.path.append("/kaggle/working/BOT-SORT/tracker")  # Add the tracker directory to Python path
from bot_sort import BoTSORT

In [ ]:
!sed -i 's/from collections.abc import Mapping, OrderedDict/from collections.abc import Mapping\nfrom collections import OrderedDict/' /kaggle/working/BOT-SORT/fast_reid/fastreid/evaluation/testing.py


In [ ]:
!mv /kaggle/input/reidsbs/mot20_sbs_S50.pth /kaggle/working/BOT-SORT/fast_reid/fastreid/mot20_sbs_S50.pth


In [ ]:
print(help(BoTSORT))


In [ ]:
# StrongSORT Configuration
strongsort_config = """
tracker_type: strongsort
track_high_thresh: 0.6
track_low_thresh: 0.05
new_track_thresh: 0.1
iou_thresh: 0.5
match_thresh: 0.95
track_buffer: 90
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 120
proximity_thresh: 0.3
appearance_thresh: 0.15
with_reid: True
reid_model: /kaggle/input/face-reid/market_bot_R50-ibn.pth
gmc_method: sparseOptFlow
"""

# Save StrongSORT configuration to strongsort.yaml
with open("/kaggle/working/strongsort.yaml", "w") as file:
    file.write(strongsort_config)

print("strongsort.yaml created successfully in /kaggle/working!")

In [ ]:
# Run StrongSORT
run_tracking("/kaggle/working/sort.yaml", "/kaggle/working/strongsort_results.txt")

In [ ]:
# Run StrongSORT
run_tracking("/kaggle/working/ocsort.yaml", "/kaggle/working/ocsort_results.txt")

In [ ]:
!pip install torchreid

In [ ]:
import torchreid
print(torchreid.__version__)

In [ ]:
!pip uninstall -y torchreid
!pip install torchreid

In [ ]:
import torch
from torchreid import models

# Load a ResNet-50 model
reid_model = models.build_model(name="resnet50", num_classes=1000, pretrained=True)
reid_model.eval()

# Load the state dictionary
state_dict = torch.load("/kaggle/input/face-reid/market_bot_R50-ibn.pth")

# Extract the model weights
model_weights = state_dict["model"]

# Load the model weights into the ReID model
reid_model.load_state_dict(model_weights)
reid_model.eval()

In [ ]:
state_dict = torch.load("/kaggle/input/face-reid/market_bot_R50-ibn.pth")
print(state_dict.keys())

In [ ]:
state_dict = torch.load("/kaggle/input/face-reid/market_bot_R50-ibn.pth")
model_weights = state_dict["model"]
print(model_weights.keys())

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

class CustomReIDModel(nn.Module):
    def __init__(self, num_classes=1000):
        super(CustomReIDModel, self).__init__()
        # Load a ResNet-50 backbone
        self.backbone = resnet50(pretrained=False)
        
        # Remove the final classification layer
        self.backbone.fc = nn.Identity()
        
        # Add a bottleneck layer (bnneck)
        self.bnneck = nn.BatchNorm1d(2048)
        self.bnneck.bias.requires_grad_(False)  # No bias for BatchNorm
        
        # Add a classifier layer
        self.classifier = nn.Linear(2048, num_classes, bias=False)
        
    def forward(self, x):
        # Extract features from the backbone
        features = self.backbone(x)
        
        # Apply bottleneck layer
        features = self.bnneck(features)
        
        # Apply classifier
        logits = self.classifier(features)
        
        return features, logits

In [ ]:
# Initialize the custom model
reid_model = CustomReIDModel(num_classes=1000)
reid_model.eval()

# Load the state dictionary
state_dict = torch.load("/kaggle/input/face-reid/market_bot_R50-ibn.pth")

# Extract the model weights
model_weights = state_dict["model"]

# Load the model weights into the custom model
reid_model.load_state_dict(model_weights, strict=False)
reid_model.eval()

In [ ]:
from torchvision import transforms
from PIL import Image

# Preprocess the detection crop
preprocess = transforms.Compose([
    transforms.Resize((256, 128)),  # Resize to the input size of the ReID model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image_crop):
    """
    Extract appearance features from an image crop using the ReID model.
    """
    image_crop = preprocess(image_crop).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features, _ = reid_model(image_crop)  # Extract features (ignore logits)
    return features

In [ ]:
from scipy.spatial.distance import cdist

# Compute appearance similarity
def compute_appearance_similarity(features1, features2):
    """
    Compute cosine similarity between two sets of features.
    """
    return 1 - cdist(features1, features2, metric="cosine")

# Combine IoU and appearance similarity
def combine_matching(iou_matrix, appearance_matrix, alpha=0.5):
    """
    Combine IoU and appearance similarity into a single matching score.
    """
    return alpha * iou_matrix + (1 - alpha) * appearance_matrix

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

class CustomReIDModel(nn.Module):
    def __init__(self, num_classes=1000):
        super(CustomReIDModel, self).__init__()
        # Load a ResNet-50 backbone
        self.backbone = resnet50(pretrained=False)
        
        # Remove the final classification layer
        self.backbone.fc = nn.Identity()
        
        # Add a bottleneck layer (bnneck)
        self.bnneck = nn.BatchNorm1d(2048)
        self.bnneck.bias.requires_grad_(False)  # No bias for BatchNorm
        
        # Add a classifier layer
        self.classifier = nn.Linear(2048, num_classes, bias=False)
        
    def forward(self, x):
        # Extract features from the backbone
        features = self.backbone(x)
        
        # Apply bottleneck layer
        features = self.bnneck(features)
        
        # Apply classifier
        logits = self.classifier(features)
        
        return features, logits

# Initialize the custom model
reid_model = CustomReIDModel(num_classes=1000)
reid_model.eval()

# Load the state dictionary
state_dict = torch.load("/kaggle/input/face-reid/market_bot_R50-ibn.pth")

# Extract the model weights
model_weights = state_dict["model"]

# Load the model weights into the custom model
reid_model.load_state_dict(model_weights, strict=False)
reid_model.eval()

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
from torchvision import transforms
from PIL import Image

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/bytetracking_ftlast_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Preprocess function for ReID
preprocess = transforms.Compose([
    transforms.Resize((256, 128)),  # Resize to the input size of the ReID model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image_crop):
    """
    Extract appearance features from an image crop using the ReID model.
    """
    image_crop = preprocess(image_crop).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features, _ = reid_model(image_crop)  # Extract features (ignore logits)
    return features

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with ByteTrack
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/bytetrack.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class (1 & 7 mapped to 1)
        cls = int(track.cls[0].item())
        if cls == 0:
            # Convert to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Extract appearance features
            image_crop = img[y:y+h, x:x+w]  # Crop the detection
            if image_crop.size == 0:
                continue  # Skip empty crops
            image_crop = Image.fromarray(cv2.cvtColor(image_crop, cv2.COLOR_BGR2RGB))
            features = extract_features(image_crop)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort tracking results by track_id first, then by frame number
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")

## BYTE 

In [ ]:
!pip install cython scipy lap motmetrics ultralytics
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip install -r requirements.txt


In [ ]:
!mkdir pretrained
!wget -O pretrained/bytetrack_x_mot20.pth.tar "https://github.com/ifzhang/ByteTrack/releases/download/v0.1.0/bytetrack_x_mot20.pth.tar"


In [ ]:
import os

weights_dir = "/kaggle/working/fast-reid/"
for root, dirs, files in os.walk(weights_dir):
    for file in files:
        print(os.path.join(root, file))  # Print all files


In [ ]:
MODEL:
  BACKBONE:
    NAME: "build_resnet_backbone"
    DEPTH: 50
  HEADS:
    NAME: "EmbeddingHead"
    IN_FEAT: 2048
    NUM_CLASSES: 751
    POOL_LAYER: "avgpool"
  META_ARCHITECTURE: "Baseline"
  PIXEL_MEAN: [123.675, 116.28, 103.53]
  PIXEL_STD: [58.395, 57.12, 57.375]

DATASETS:
  NAMES: ("Market1501",)
  TESTS: ("Market1501",)

DATALOADER:
  NUM_WORKERS: 8

SOLVER:
  IMS_PER_BATCH: 64
  BASE_LR: 0.00035
  MAX_ITER: 12000
  STEPS: [4000, 8000]
  CHECKPOINT_PERIOD: 2000

TEST:
  IMS_PER_BATCH: 128
  EVAL_PERIOD: 2000


In [ ]:
!wget -O /kaggle/working/fast-reid/model_final.pth https://github.com/JDAI-CV/fast-reid/releases/download/v0.1.1/market_mgn_R50-ibn.pth


In [ ]:
import torch
import fastreid
from fastreid.config import get_cfg
from fastreid.modeling import build_model
from fastreid.utils.checkpoint import Checkpointer
from torchvision import transforms
import numpy as np

# Load FastReID Model
def load_reid_model(config_path, weights_path):
    cfg = get_cfg()
    cfg.merge_from_file(config_path)
    model = build_model(cfg)
    Checkpointer(model).load(weights_path)
    model.eval()
    return model

# Extract Features for ReID
def extract_features(model, img):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 128)),  # Standard ReID input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(img_tensor)
    return features.cpu().numpy()

# Compute Cosine Similarity
def cosine_similarity(feat1, feat2):
    return np.dot(feat1, feat2.T) / (np.linalg.norm(feat1) * np.linalg.norm(feat2))

# Match ReID Features to Improve Tracking
def match_reid_tracks(existing_tracks, new_detections, model):
    matched_tracks = {}
    for track_id, track_feat in existing_tracks.items():
        best_match = None
        best_score = 0.0
        for det_id, det_img in new_detections.items():
            det_feat = extract_features(model, det_img)
            score = cosine_similarity(track_feat, det_feat)
            if score > best_score:
                best_match = det_id
                best_score = score
        if best_match and best_score > 0.7:  # Threshold to avoid mismatches
            matched_tracks[track_id] = best_match
    return matched_tracks

# Load ReID model
reid_model = load_reid_model("/kaggle/working/fast-reid/configs/Base-MGN.yml", "/kaggle/working/fast-reid/model_final.pth")


In [ ]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from fastreid.config import get_cfg
from fastreid.modeling import build_model
from fastreid.utils.checkpoint import Checkpointer
from torchvision import transforms

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/bytetrack_results.txt"

# Load YOLO Model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Load FastReID Model
def load_reid_model(config_path, weights_path):
    cfg = get_cfg()
    cfg.merge_from_file(config_path)
    model = build_model(cfg)
    Checkpointer(model).load(weights_path)
    model.eval().cuda()
    return model

reid_model = load_reid_model("/kaggle/working/fast-reid/configs/Base-MGN.yml", "/kaggle/working/fast-reid/model_final.pth")

# Transform for FastReID
reid_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Batch Feature Extraction for ReID
def extract_reid_features_batch(model, images):
    if not images:
        return {}
    
    imgs_tensor = torch.stack([reid_transform(img) for img in images]).cuda()
    with torch.no_grad():
        features = model(imgs_tensor)
    
    return {i: features[i].cpu().numpy().flatten() for i in range(len(images))}

# Compute Cosine Similarity
def cosine_similarity(feat1, feat2):
    return np.dot(feat1, feat2) / (np.linalg.norm(feat1) * np.linalg.norm(feat2))

# ReID Track Matching
def match_reid_tracks(existing_tracks, new_detections):
    matched_tracks = {}
    new_ids, new_feats = list(new_detections.keys()), list(new_detections.values())

    if not new_feats:
        return matched_tracks

    # Batch extract new features
    new_features = extract_reid_features_batch(reid_model, new_feats)

    for track_id, track_feat in existing_tracks.items():
        best_match, best_score = None, -1.0
        for det_id, det_feat in new_features.items():
            score = cosine_similarity(track_feat, det_feat)
            if score > best_score:
                best_match, best_score = det_id, score

        if best_match and best_score > 0.7:  # Threshold to reduce mismatches
            matched_tracks[track_id] = best_match

    return matched_tracks

# Initialize tracking memory
track_features = {}
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])
    img_path = os.path.join(test_images_path, img_name)
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue

    # Run YOLO tracking
    results = model2.track(img, persist=True, tracker="/kaggle/working/bytetrack.yaml", imgsz=(704,704))[0]

    new_detections = {}
    cropped_images = []
    track_ids = []

    for track in results.boxes:
        x, y, w, h = map(int, track.xywh[0].cpu().numpy())
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = round(track.conf[0].item(), 4)
        cls = int(track.cls[0].item())

        if cls == 0:  # Only process people
            cropped_person = img[y:y+h, x:x+w]
            cropped_images.append(cropped_person)
            track_ids.append(track_id)

            tracking_results.append([frame_number, track_id, x, y, w, h, conf, 1, 1])

    # Batch extract new ReID features
    if cropped_images:
        extracted_features = extract_reid_features_batch(reid_model, cropped_images)
        for idx, track_id in enumerate(track_ids):
            track_features[track_id] = extracted_features[idx]

    # Match lost tracks using ReID
    matched_tracks = match_reid_tracks(track_features, new_detections)

    # Update track IDs with ReID-matched IDs
    for old_id, new_id in matched_tracks.items():
        for track in tracking_results:
            if track[1] == old_id:
                track[1] = new_id

# Save tracking results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):  # Sort by ID first, then frame
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")


In [ ]:
%cd /kaggle/working/fast-reid


In [ ]:
!find . -type f -name "*.py" -exec sed -i 's/from collections.abc import Mapping, OrderedDict/from collections.abc import Mapping\nfrom collections import OrderedDict/g' {} +


In [ ]:
!grep -r "from collections.abc import OrderedDict" /kaggle/working/BOT-SORT/fast_reid/


In [ ]:
!grep -r "from collections.abc import OrderedDict" .


In [ ]:
ls

In [ ]:
!pip install filterpy scikit-learn
!pip install faiss-gpu


In [ ]:
!find /kaggle/working/BOT-SORT/fast_reid/fastreid -type f -name "*.py" -exec sed -i 's/from collections import Mapping/from collections.abc import Mapping/g' {} +


In [ ]:
!git clone https://github.com/JDAI-CV/fast-reid.git
%cd fast-reid
!pip install -r requirements.txt


In [ ]:
!pip install yacs


In [ ]:
import os
import cv2
import torch
import numpy as np
import fastreid
import pandas as pd
from ultralytics import YOLO
from fastreid.config import get_cfg
from fastreid.modeling import build_model
from fastreid.utils.checkpoint import Checkpointer
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter

# Load the YOLO model
model2 = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")

# Load the ReID model
def load_reid_model(config_path, weights_path):
    cfg = get_cfg()
    cfg.merge_from_file(config_path)
    model = build_model(cfg)
    Checkpointer(model).load(weights_path)
    model.eval().cuda()
    return model

reid_model = load_reid_model("/kaggle/working/fast-reid/configs/Base-MGN.yml", "/kaggle/working/fast-reid/model_final.pth")

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/bytetracking_ftlast_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704
target_height = 704

# Tracking results storage
tracking_results = []
track_features = {}  # Store features for each track ID
track_last_frame = {}  # Store the last frame number for each track ID
track_counter = 0  # Counter for assigning new IDs
max_unmatched_frames = 30  # Maximum frames to keep a track alive without matching

# Kalman filter setup
def create_kalman_filter():
    kf = KalmanFilter(dim_x=7, dim_z=4)
    kf.F = np.array([[1, 0, 0, 0, 1, 0, 0],
                     [0, 1, 0, 0, 0, 1, 0],
                     [0, 0, 1, 0, 0, 0, 1],
                     [0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0]])
    kf.P *= 1000  # Initial uncertainty
    kf.R = np.eye(4) * 5  # Reduced measurement noise
    kf.Q = np.eye(7) * 0.5  # Increased process noise
    return kf

track_kalman_filters = {}  # Store Kalman filters for each track ID

# Feature aggregation using exponential moving average (EMA)
def update_feature_ema(track_id, new_feature, alpha=0.8):  # Reduced alpha
    if track_id in track_features:
        track_features[track_id] = alpha * track_features[track_id] + (1 - alpha) * new_feature
    else:
        track_features[track_id] = new_feature

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])
    img_path = os.path.join(test_images_path, img_name)

    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/bytetrack.yaml")[0]

    current_frame_features = {}
    current_frame_boxes = {}

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()

        cls = int(track.cls[0].item())
        if cls == 0:
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)

            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            person_img = img[y:y+h, x:x+w]
            if person_img.size == 0:
                continue

            person_img_resized = cv2.resize(person_img, (128, 256))
            person_img_tensor = torch.from_numpy(person_img_resized).permute(2, 0, 1).float().div(255.0).unsqueeze(0).cuda()

            with torch.no_grad():
                reid_features = reid_model(person_img_tensor).cpu().numpy()

            current_frame_features[track_id] = reid_features
            current_frame_boxes[track_id] = (x, y, w, h, conf)

    # Match tracks using Hungarian algorithm
    if track_features:
        track_ids = list(track_features.keys())
        current_ids = list(current_frame_features.keys())

        cost_matrix = np.zeros((len(track_ids), len(current_ids)))
        for i, tid in enumerate(track_ids):
            for j, cid in enumerate(current_ids):
                cost_matrix[i, j] = cdist(track_features[tid], current_frame_features[cid], metric='cosine')

        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matched_pairs = list(zip(row_ind, col_ind))

        matched_tracks = set()
        matched_detections = set()

        for i, j in matched_pairs:
            if cost_matrix[i, j] < 0.5:  # Relaxed threshold for matching
                track_id = track_ids[i]
                current_id = current_ids[j]
                update_feature_ema(track_id, current_frame_features[current_id])  # Update features using EMA
                track_last_frame[track_id] = frame_number  # Update last frame number
                x, y, w, h, conf = current_frame_boxes[current_id]
                tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])
                matched_tracks.add(track_id)
                matched_detections.add(current_id)

        # Handle unmatched tracks
        for tid in track_ids:
            if tid not in matched_tracks:
                # Remove tracks that are unmatched for too many frames
                if frame_number - track_last_frame[tid] > max_unmatched_frames:
                    del track_features[tid]
                    del track_last_frame[tid]
                    if tid in track_kalman_filters:
                        del track_kalman_filters[tid]

        # Handle unmatched detections (assign new IDs)
        for cid in current_ids:
            if cid not in matched_detections:
                track_counter += 1
                update_feature_ema(track_counter, current_frame_features[cid])  # Initialize features using EMA
                track_last_frame[track_counter] = frame_number  # Set last frame number
                x, y, w, h, conf = current_frame_boxes[cid]
                tracking_results.append([frame_number, track_counter, x, y, w, h, round(conf, 4), 1, 1])
    else:
        # First frame, assign new IDs
        for track_id, reid_features in current_frame_features.items():
            track_counter += 1
            update_feature_ema(track_counter, reid_features)  # Initialize features using EMA
            track_last_frame[track_counter] = frame_number  # Set last frame number
            x, y, w, h, conf = current_frame_boxes[track_id]
            tracking_results.append([frame_number, track_counter, x, y, w, h, round(conf, 4), 1, 1])

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")

In [ ]:
import sys
sys.path.append("/kaggle/working/fast-reid")  # Adjust the path if necessary


In [ ]:
import fastreid
print("FastReID successfully imported!")


In [ ]:

!pip install git+https://github.com/mikel-brostrom/BoT-SORT.git


## From ZERO TO HERO

In [ ]:
!pip install ultralytics lap thop opencv-python-headless supervision
!pip install fastreid torchreid


In [ ]:
import torch
import cv2
import os
import numpy as np
import supervision as sv
from ultralytics import YOLO
from pathlib import Path


In [ ]:
DATASET_PATH = Path("/kaggle/input/tracking/tracking/tracking/test/01/img1") # Update with correct path
assert DATASET_PATH.exists(), "Dataset not found! Upload MOT20-01."


In [ ]:
# Load YOLOv8 model (can use yolov8m.pt or a fine-tuned one)
model = YOLO("/kaggle/input/model-trial/pytorch/default/3/best (1).pt")  # Change this if using your fine-tuned model


In [ ]:
import matplotlib.pyplot as plt
import cv2

# Test on a single image
image_path = str(DATASET_PATH/ "000001.jpg")  # Ensure correct path
results = model(image_path)

# Show results using Matplotlib
for r in results:
    im_array = r.plot()  # YOLOv8 outputs BGR image

    # Convert BGR to RGB
    im_array = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)

    # Resize for better visualization
    im_resized = cv2.resize(im_array, (800, 400))

    # Display using Matplotlib
    plt.figure(figsize=(10, 5))
    plt.imshow(im_resized)
    plt.axis("off")
    plt.show()


In [ ]:
!pip install strongsort supervision torchreid


In [ ]:
# from ultralytics.trackers.byte_tracker import BYTETracker
# from ultralytics.trackers.bot_sort import BOTSORT
# from types import SimpleNamespace

# class BOTSORT(BYTETracker):
#     def __init__(self):
#         """
#         Initialize YOLOv8 object with ReID module and GMC algorithm.
#         """
#         # Create a simple args object with required attributes
#         args = SimpleNamespace(track_buffer=30)  # Add other required attributes if needed
#         super().__init__(args=args, frame_rate=25)  # Pass the args object

#         # Assign the provided parameters
#         self.with_reid = True
#         self.appearance_thresh = 0.25
#         self.iou_threshold = 0.3
#         self.max_age = 30
#         self.min_hits = 3
#         self.model_weights = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"
#         self.proximity_thresh = 0.5
#         self.track_buffer = 30
#         self.tracker_type = "botsort"
#         self.track_high_thresh = 0.25
#         self.track_low_thresh = 0.25
#         self.track_buffer = 30
#         self.fuse_score = True
#         self.new_track_thresh = 0.25
#         self.match_thresh = 0.8

#         # Initialize additional necessary attributes
#         self.gmc_method = "sparseOptFlow"  # Global Motion Compensation

In [ ]:
!pip install -U ultralytics

In [ ]:
!git clone https://github.com/NirAharon/BoT-SORT.git
%cd BoT-SORT
!pip install -r requirements.txt

## New Model

In [ ]:
import torch

# Load full model
ckpt = torch.load('/kaggle/input/trackingyolo/pytorch/default/5/best.pt', map_location='cpu')

# Remove optimizer states
if 'optimizer' in ckpt:
    del ckpt['optimizer']

# Save the smaller model
torch.save(ckpt, 'best_stripped.pt')


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 18.6 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=ac37b21b2992522f5f467cec01ebc67e94dd2b99d9adaf321979daf3b745b55f
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [26]:
!pip install torchreid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144325 sha256=6c9b87541e38800b9fabc47af7fd705c871523503da07e53474b90df0d879195
  Stored in directory: /root/.cache/pip/wheels/bb/2d/36/816a48465cefd3e58be0317648a4c52ce39ae817f935212099
Successfully built torchreid


## Model without reid 67.99%

In [3]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Load the YOLO model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/modeltrackingresults.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []
track_history = {}  # Store previous frames' tracking data for smoother tracking

# Optimized confidence & IOU thresholds
iou_threshold = 0.6  # Slightly increased for better tracking continuity
conf_threshold = 0.22  # Lowered to retain potential detections

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with optimized parameters
    results = model2.track(img_resized, persist=True, iou=iou_threshold, conf=conf_threshold)[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None and len(track.id) > 0 else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class
        cls = int(track.cls[0].item())
        if cls == 0:
            # Scale back to original image size
            scale_x = original_width / target_width
            scale_y = original_height / target_height

            x_center *= scale_x
            y_center *= scale_y
            w *= scale_x
            h *= scale_y

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Smooth ID handling: prevent flickering IDs by checking history
            if track_id in track_history and abs(track_history[track_id][0] - x_center) < 50:
                track_id = track_history[track_id][1]  # Keep previous ID

            track_history[track_id] = (x_center, track_id)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Keep only valid track IDs (removes short-lived detections)
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Remove short-lived tracks (less than 5 frames for more stable tracking)
min_track_length = 20
track_id_data = {k: v for k, v in track_id_data.items() if len(v) >= min_track_length}

# Save results in MOT format using your original sorting
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.4s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 704x704 40 persons, 43.1ms
Speed: 7.3ms preprocess, 43.1ms inference, 270.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 39 persons, 43.2ms
Speed: 3.5ms preprocess, 43.2ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 40 persons, 43.1ms
Speed: 2.3ms preprocess, 43.1ms inference, 1.5ms postprocess per image at sh

## MODEL WITH REID

In [34]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
from torchreid import models, utils

# Load the YOLO model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/modeltrackingresults.txt"
reid_model_path = "/kaggle/input/reidsbs/mot20_sbs_S50.pth"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []
track_history = {}  # Store previous frames' tracking data for smoother tracking
track_features = {}  # Store appearance features for each track ID

# Optimized confidence & IOU thresholds
iou_threshold = 0.6  # Slightly increased for better tracking continuity
conf_threshold = 0.22  # Lowered to retain potential detections

# Load the ReID model
reid_model = models.build_model(name="osnet_x1_0", num_classes=1000)
utils.load_pretrained_weights(reid_model, reid_model_path)
reid_model.eval()

# Function to extract features using the ReID model
def extract_features(img, bbox):
    x, y, w, h = bbox
    crop = img[y:y+h, x:x+w]
    crop = cv2.resize(crop, (128, 256))  # Resize to ReID input size
    crop = torch.from_numpy(crop).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    with torch.no_grad():
        features = reid_model(crop)
    return features.cpu().numpy()

# Function to compute cosine similarity
def cosine_similarity(feat1, feat2):
    feat1 = feat1.flatten()  # Flatten to 1D array
    feat2 = feat2.flatten()  # Flatten to 1D array
    return np.dot(feat1, feat2) / (np.linalg.norm(feat1) * np.linalg.norm(feat2))

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with optimized parameters
    results = model2.track(img_resized, persist=True, iou=iou_threshold, conf=conf_threshold)[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None and len(track.id) > 0 else -1  # Ensure track_id persists
        conf = track.conf[0].item()

        # Keep only person class
        cls = int(track.cls[0].item())
        if cls == 0:
            # Scale back to original image size
            scale_x = original_width / target_width
            scale_y = original_height / target_height

            x_center *= scale_x
            y_center *= scale_y
            w *= scale_x
            h *= scale_y

            # Convert to top-left format
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w = int(w)
            h = int(h)

            # Extract appearance features using ReID model
            bbox = (x, y, w, h)
            features = extract_features(img, bbox)

            # Match features with existing tracks
            best_match_id = None
            best_similarity = -1

            for existing_id, existing_features in track_features.items():
                similarity = cosine_similarity(features, existing_features)
                if similarity > best_similarity:
                    best_similarity = similarity
                    best_match_id = existing_id

            # Re-assign ID if similarity is above a threshold
            if best_similarity > 0.95:  # Threshold for matching
                track_id = best_match_id

            # Update track history and features
            track_history[track_id] = (x_center, track_id)
            track_features[track_id] = features

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Keep only valid track IDs (removes short-lived detections)
track_id_data = {}

for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Remove short-lived tracks (less than 5 frames for more stable tracking)
min_track_length = 20
track_id_data = {k: v for k, v in track_id_data.items() if len(v) >= min_track_length}

# Save results in MOT format using your original sorting
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


/usr/local/lib/python3.10/dist-packages/torchreid/reid/utils/torchtools.py:297: UserWarning: The pretrained weights "/kaggle/input/reidsbs/mot20_sbs_S50.pth" cannot be loaded, please check the key names manually (** ignored and continue **)
  warnings.warn(



0: 704x704 40 persons, 43.1ms
Speed: 1.9ms preprocess, 43.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 39 persons, 43.1ms
Speed: 2.1ms preprocess, 43.1ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 40 persons, 43.2ms
Speed: 2.0ms preprocess, 43.2ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 38 persons, 43.1ms
Speed: 2.0ms preprocess, 43.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 35 persons, 43.1ms
Speed: 1.9ms preprocess, 43.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 39 persons, 43.1ms
Speed: 2.0ms preprocess, 43.1ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 40 persons, 43.1ms
Speed: 1.7ms preprocess, 43.1ms inference, 1.8ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 41 persons, 43.1ms
Speed: 1.8ms preprocess, 43.1ms inference, 1.4ms postprocess per image at

In [6]:
import shutil

# Define folder paths
visualization_output = "/kaggle/working/modeltracking_output"
zip_path = "/kaggle/working/visualized_modeltrack_output.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', visualization_output)

print(f"Zipped visualization folder saved at: {zip_path}")


Zipped visualization folder saved at: /kaggle/working/visualized_modeltrack_output.zip


In [5]:
!pip install motmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 6.0 MB/s eta 0:00:00


In [8]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/modeltrackingresults.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


         mota      motp      idf1  precision    recall
acc  0.679914  0.325606  0.694606   0.911306  0.759936


## Byte

In [31]:
import yaml

# Define the path to your bytetrack.yaml file
file_path = "/kaggle/working/bytetrack.yaml"

# Load the existing YAML file
with open(file_path, "r") as file:
    bytetrack_config = yaml.safe_load(file)
# tracking_0.6_0.1_0.3_0.85_30_0.2_True.txt

# "tracker_type": "bytetrack",
#         "track_high_thresh": params[0],
#         "track_low_thresh": params[1],
#         "iou_thresh": params[2],
#         "match_thresh": params[3],
#         "track_buffer": params[4],
#         "new_track_thresh": params[5],
#         "fuse_score": params[6],


# Modify the values with optimized settings
bytetrack_config["tracker_type"] = "bytetrack"
bytetrack_config["track_high_thresh"] = 0.42  # Higher confidence filtering for stable tracks ##effect
bytetrack_config["track_low_thresh"] = 0.1  # Keep lower-confidence detections to maintain recall ##no
bytetrack_config["new_track_thresh"] = 0.2  # Prevent frequent new ID assignments #no
bytetrack_config["iou_thresh"] = 0.58  # Higher IoU helps in tracking objects with slight movement #no
bytetrack_config["match_thresh"] = 0.78  # Reduce ID switches by making matching stricter #bigeffect
bytetrack_config["track_buffer"] = 30  # Increase buffer to maintain identity over more frames #small 
bytetrack_config["fuse_score"] = True  # Enable score fusion for better tracking #true better
bytetrack_config["frame_rate"] = 25  # Ensure tracking is adjusted for 25 FPS videos
bytetrack_config["det_thresh"] = 0.5  # Detection confidence threshold #no effect
bytetrack_config["lost_track_thresh"] = 10  # Use confidence fusion for better tracking #no effect
# Save the updated YAML file
with open(file_path, "w") as file:
    yaml.dump(bytetrack_config, file, default_flow_style=False)

print("bytetrack.yaml has been updated successfully with optimized settings!")


bytetrack.yaml has been updated successfully with optimized settings!


In [32]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 fine-tuned model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/bytetracking_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with ByteTrack
    results = model2.track(img_resized, persist=True, iou=0.58, conf=0.25,tracker="/kaggle/working/bytetrack.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        # Process only persons (class 0)
        if cls == 0:
            # Convert coordinates to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w, h = int(w), int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort and save results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")



0: 704x704 38 persons, 43.1ms
Speed: 2.1ms preprocess, 43.1ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 38 persons, 42.2ms
Speed: 2.1ms preprocess, 42.2ms inference, 1.5ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 38 persons, 30.1ms
Speed: 2.1ms preprocess, 30.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 35 persons, 30.1ms
Speed: 2.0ms preprocess, 30.1ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 34 persons, 30.0ms
Speed: 1.9ms preprocess, 30.0ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 36 persons, 29.7ms
Speed: 2.0ms preprocess, 29.7ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 37 persons, 29.7ms
Speed: 2.1ms preprocess, 29.7ms inference, 1.8ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 39 persons, 28.8ms
Speed: 2.3ms preprocess, 28.8ms inference, 1.5ms postprocess per image at

In [33]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/bytetracking_results.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


        mota      motp      idf1  precision    recall
acc  0.66989  0.327519  0.626394   0.918092  0.743629


In [55]:
import pandas as pd
import ast

# Load MOT tracking results
def load_mot_results(mot_file):
    tracking_data = {}
    with open(mot_file, 'r') as f:
        for line in f:
            frame_id, track_id, x, y, w, h, conf, *_ = map(float, line.strip().split(','))
            if int(frame_id) not in tracking_data:
                tracking_data[int(frame_id)] = []
            tracking_data[int(frame_id)].append({
                "tracked_id": track_id, "x": int(x), "y": int(y),
                "w": int(w), "h": int(h), "confidence": conf
            })
    return tracking_data

# Load submission template
def load_submission_template(submission_file):
    df = pd.read_csv(submission_file)
    return df

# Process and fill submission file
def fill_submission_file(mot_file, submission_file, output_file):
    tracking_data = load_mot_results(mot_file)
    df = load_submission_template(submission_file)
    
    for i in range(len(df)):
        if df.loc[i, "objective"] == "tracking":
            frame_id = int(df.loc[i, "frame"])  # Get frame from submission template
            if frame_id in tracking_data:
                df.loc[i, "objects"] = str(tracking_data[frame_id])  # Fill tracking results
        elif df.loc[i, "objective"] == "face_reid":
            df.loc[i, "frame"] = -1  # Ensure face_reid has frame -1
            df.loc[i, "objects"] = str(ast.literal_eval(df.loc[i, "objects"]))  # Keep original

    # Save final submission file
    df.to_csv(output_file, index=False)
    print(f"Submission file saved as: {output_file}")

# Example usage
mot_file = "/kaggle/working/bytetracking_results.txt"
submission_file = "/kaggle/input/fawrysub1/sub1.csv"
output_file = "/kaggle/working/mv2.csv"

fill_submission_file(mot_file, submission_file, output_file)


Submission file saved as: /kaggle/working/mv2.csv


## Botsort

In [25]:
# # BoT-SORT Configuration
botsort_config = """
tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.1
new_track_thresh: 0.2
iou_thresh: 0.3
match_thresh: 0.85
track_buffer: 30
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 50
proximity_thresh: 0.5
appearance_thresh: 0.25
with_reid: False
gmc_method: sparseOptFlow
"""

# OC-SORT Configuration
ocsort_config = """
tracker_type: ocsort
track_high_thresh: 0.6
track_low_thresh: 0.1
new_track_thresh: 0.2
iou_thresh: 0.3
match_thresh: 0.85
track_buffer: 30
fuse_score: True
frame_rate: 25
det_thresh: 0.5
lost_track_thresh: 50
"""

# Save BoT-SORT configuration to botsort.yaml
with open("/kaggle/working/botsort.yaml", "w") as file:
    file.write(botsort_config)

# Save OC-SORT configuration to ocsort.yaml
with open("/kaggle/working/ocsort.yaml", "w") as file:
    file.write(ocsort_config)

print("YAML files created successfully in /kaggle/working!")

YAML files created successfully in /kaggle/working!


In [23]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 fine-tuned model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/bottracking_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with ByteTrack
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/botsort.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        # Process only persons (class 0)
        if cls == 0:
            # Convert coordinates to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w, h = int(w), int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort and save results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")



0: 704x704 18 persons, 43.1ms
Speed: 1.8ms preprocess, 43.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 18 persons, 43.1ms
Speed: 1.7ms preprocess, 43.1ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 19 persons, 29.5ms
Speed: 1.9ms preprocess, 29.5ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 20 persons, 29.4ms
Speed: 1.7ms preprocess, 29.4ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 29.3ms
Speed: 1.8ms preprocess, 29.3ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 27.4ms
Speed: 1.7ms preprocess, 27.4ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 26.5ms
Speed: 1.8ms preprocess, 26.5ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 23 persons, 27.4ms
Speed: 1.6ms preprocess, 27.4ms inference, 1.1ms postprocess per image at

In [24]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/bottracking_results.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


         mota      motp      idf1  precision    recall
acc  0.593299  0.320313  0.576551   0.931008  0.646944


## OCSORT

In [17]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 fine-tuned model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/ocsorttracking_results.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 tracking with ByteTrack
    results = model2.track(img_resized, persist=True, tracker="/kaggle/working/botsort.yaml")[0]  

    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        track_id = int(track.id[0].item()) if track.id is not None else -1
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())

        # Process only persons (class 0)
        if cls == 0:
            # Convert coordinates to original image size
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            w, h = int(w), int(h)

            # Save tracking result in MOT format
            tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

            # Draw bounding boxes on image
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort and save results in MOT format
with open(output_mot_path, "w") as f:
    for row in sorted(tracking_results, key=lambda x: (x[1], x[0])):
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")



0: 704x704 18 persons, 43.0ms
Speed: 1.8ms preprocess, 43.0ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 18 persons, 43.0ms
Speed: 1.7ms preprocess, 43.0ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 19 persons, 28.8ms
Speed: 2.0ms preprocess, 28.8ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 20 persons, 27.9ms
Speed: 1.6ms preprocess, 27.9ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 27.8ms
Speed: 1.8ms preprocess, 27.8ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 27.8ms
Speed: 1.8ms preprocess, 27.8ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 21 persons, 28.7ms
Speed: 1.8ms preprocess, 28.7ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 23 persons, 27.8ms
Speed: 1.9ms preprocess, 27.8ms inference, 1.1ms postprocess per image at

In [18]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/ocsorttracking_results.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


         mota      motp      idf1  precision   recall
acc  0.593878  0.320855  0.581676   0.929228  0.64877


## BoostTrack algo

In [8]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=c124f77d52eab70dfddbd9ab50da7f3abc98932cc2484a7a80e479c7866b89b3
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [9]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment

# Load the YOLO model
model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
gt_file_path = "/kaggle/input/testgt/gt.txt"
output_mot_path = "/kaggle/working/modeltrackingresults.txt"

# Resize parameters
original_width = 1920
original_height = 1080
target_width = 704  # YOLOv8 input size
target_height = 704

# Tracking results storage
tracking_results = []
previous_tracks = {}
next_track_id = 0

# IoU Calculation
def iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    return inter_area / (box1_area + box2_area - inter_area)

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])  # Extract frame number
    img_path = os.path.join(test_images_path, img_name)

    # Load and resize image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_resized = cv2.resize(img, (target_width, target_height))

    # Run YOLOv8 detection (without built-in tracking)
    results = model2(img_resized)[0]

    detections = []
    for track in results.boxes:
        x_center, y_center, w, h = track.xywh[0].cpu().numpy()
        conf = track.conf[0].item()
        cls = int(track.cls[0].item())
        if cls == 0:
            x_center *= (original_width / target_width)
            y_center *= (original_height / target_height)
            w *= (original_width / target_width)
            h *= (original_height / target_height)
            x = int(x_center - w / 2)
            y = int(y_center - h / 2)
            detections.append([x, y, w, h, conf])

    # Match detections with previous tracks using Hungarian Algorithm
    if previous_tracks:
        track_ids = list(previous_tracks.keys())
        track_boxes = np.array([previous_tracks[t] for t in track_ids])
        detection_boxes = np.array([d[:4] for d in detections])
        cost_matrix = np.zeros((len(track_boxes), len(detection_boxes)))
        for i, tbox in enumerate(track_boxes):
            for j, dbox in enumerate(detection_boxes):
                cost_matrix[i, j] = 1 - iou(tbox, dbox)
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        assigned_tracks = {}
        assigned_detections = set()
        for r, c in zip(row_ind, col_ind):
            if cost_matrix[r, c] < 0.5:  # IoU threshold
                assigned_tracks[track_ids[r]] = detections[c][:4]
                assigned_detections.add(c)

        # Assign new IDs to unassigned detections
        for i, det in enumerate(detections):
            if i not in assigned_detections:
                assigned_tracks[next_track_id] = det[:4]
                next_track_id += 1

        previous_tracks = assigned_tracks
    else:
        for det in detections:
            previous_tracks[next_track_id] = det[:4]
            next_track_id += 1

    # Store tracking results
    for track_id, box in previous_tracks.items():
        x, y, w, h = box
        conf = next((d[4] for d in detections if d[:4] == box), 1.0)  # Get confidence from detections
        tracking_results.append([frame_number, track_id, x, y, w, h, round(conf, 4), 1, 1])

# Sort tracking results by track_id first, then by frame number
track_id_data = {}
for row in tracking_results:
    frame_number, track_id = row[0], row[1]
    if track_id not in track_id_data:
        track_id_data[track_id] = []
    track_id_data[track_id].append(row)

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for track_id in sorted(track_id_data.keys()):  # Sort by track_id first
        for row in sorted(track_id_data[track_id], key=lambda x: x[0]):  # Sort frames within each ID
            f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")



0: 704x704 42 persons, 43.1ms
Speed: 2.0ms preprocess, 43.1ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 46 persons, 43.1ms
Speed: 2.2ms preprocess, 43.1ms inference, 1.7ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 47 persons, 39.7ms
Speed: 2.3ms preprocess, 39.7ms inference, 1.7ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 40 persons, 39.8ms
Speed: 2.2ms preprocess, 39.8ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 41 persons, 38.2ms
Speed: 2.1ms preprocess, 38.2ms inference, 1.7ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 44 persons, 38.2ms
Speed: 2.0ms preprocess, 38.2ms inference, 1.6ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 48 persons, 38.1ms
Speed: 2.0ms preprocess, 38.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 47 persons, 38.2ms
Speed: 2.3ms preprocess, 38.2ms inference, 1.7ms postprocess per image at

In [4]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/input/gttrials/tracking_results (3).txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)


         mota      motp      idf1  precision    recall
acc  0.556363  0.330231  0.606879   0.776893  0.790946


In [11]:
!git clone https://github.com/vukasin-stanojevic/BoostTrack.git
%cd BoostTrack

Cloning into 'BoostTrack'...
remote: Enumerating objects: 1643, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1643 (delta 90), reused 59 (delta 59), pack-reused 1451 (from 1)
Receiving objects: 100% (1643/1643), 129.81 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (339/339), done.
/kaggle/working/BoostTrack


## YOLOX

In [3]:
!pip install yolox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of yolox to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 2.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.



In [5]:
!pip install -U git+https://github.com/Megvii-BaseDetection/YOLOX.git


  Cloning https://github.com/Megvii-BaseDetection/YOLOX.git to /tmp/pip-req-build-tz8qw2wp
  Running command git clone --filter=blob:none --quiet https://github.com/Megvii-BaseDetection/YOLOX.git /tmp/pip-req-build-tz8qw2wp
  Resolved https://github.com/Megvii-BaseDetection/YOLOX.git to commit d872c71bf63e1906ef7b7bb5a9d7a529c7a59e6a
  Preparing metadata (setup.py) ... done
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.5 MB/s eta 0:00:00
  Created wheel for yolox: filename=yolox-0.3.0-cp310-cp310-linux_x86_64.whl size=1538296 sha256=318cf9dd73f21a6d83a3686abdab66783bb0bfb6f5765e1f226b8e2e2f6aefac
  Stored in directory: /tmp/pip-ephem-wheel-cache-8cfj3_d6/wheels/34/59/cd/6e6affe0835b4ebf5b53f47f913fcff04e2a03042616955c9e
Successfully built yolox


In [16]:
import os
import cv2
import torch
import numpy as np
from yolox.exp import get_exp
from yolox.data.data_augment import preproc
from yolox.utils import fuse_model, postprocess
from YOLOX.yolox.tracker.byte_tracker import BYTETracker
from yolox.utils.visualize import vis

# Load YOLOX model
exp = get_exp(None, "yolox_s")  # Change "yolox_s" to your model size
model = exp.get_model()
ckpt = torch.load("/kaggle/input/yoloxs/yolox_s.pth", map_location="cpu")
model.load_state_dict(ckpt["model"])
model.eval()
fuse_model(model)
tracker = BYTETracker()

# Paths
test_images_path = "/kaggle/input/tracking/tracking/tracking/test/01/img1"
output_mot_path = "/kaggle/working/modeltrackingresults.txt"

# Tracking results storage
tracking_results = []

# Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])
    img_path = os.path.join(test_images_path, img_name)
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img, ratio = preproc(img, exp.test_size, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    img_tensor = torch.from_numpy(img).unsqueeze(0).float()
    
    with torch.no_grad():
        outputs = model(img_tensor)
        detections = postprocess(outputs[0], num_classes=1, conf_thre=0.25, nms_thre=0.45)
    
    if detections is not None:
        tracked_objects = tracker.update(detections.cpu().numpy(), exp.test_size, ratio)
        
        for track in tracked_objects:
            track_id = int(track.track_id)
            x, y, w, h = map(int, track.tlwh)
            conf = round(track.score, 4)
            
            tracking_results.append([frame_number, track_id, x, y, w, h, conf, 1, 1])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, f"ID {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sort tracking results: first by Frame Number, then by Track ID
tracking_results.sort(key=lambda x: (x[1], x[0]))

# Save results in MOT format
with open(output_mot_path, "w") as f:
    for row in tracking_results:
        f.write(",".join(map(str, row)) + "\n")

print(f"Tracking results saved to {output_mot_path}")


ModuleNotFoundError: No module named 'yolox.tracker'

In [7]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
!pip install -e .


Cloning into 'YOLOX'...
remote: Enumerating objects: 1928, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1928 (delta 2), reused 0 (delta 0), pack-reused 1916 (from 2)
Receiving objects: 100% (1928/1928), 7.55 MiB | 27.13 MiB/s, done.
Resolving deltas: 100% (1148/1148), done.
/kaggle/working/YOLOX
Obtaining file:///kaggle/working/YOLOX
  Preparing metadata (setup.py) ... done
  Attempting uninstall: yolox
    Found existing installation: yolox 0.3.0
    Uninstalling yolox-0.3.0:
      Successfully uninstalled yolox-0.3.0
  Running setup.py develop for yolox


In [9]:
!pip install -e /kaggle/working/YOLOX


Obtaining file:///kaggle/working/YOLOX
  Preparing metadata (setup.py) ... done
  Attempting uninstall: yolox
    Found existing installation: yolox 0.3.0
    Uninstalling yolox-0.3.0:
      Successfully uninstalled yolox-0.3.0
  Running setup.py develop for yolox


In [10]:
!ls /kaggle/working/YOLOX/yolox


core  evaluators  __init__.py  models	    tools
data  exp	  layers       __pycache__  utils


In [11]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip install -e .


Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007 (from 1)
Receiving objects: 100% (2007/2007), 79.60 MiB | 47.61 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
/kaggle/working/YOLOX/ByteTrack
Obtaining file:///kaggle/working/YOLOX/ByteTrack
  Preparing metadata (setup.py) ... done
  Attempting uninstall: yolox
    Found existing installation: yolox 0.3.0
    Uninstalling yolox-0.3.0:
      Successfully uninstalled yolox-0.3.0
  Running setup.py develop for yolox


In [13]:
!cp -r /kaggle/working/YOLOX/ByteTrack/yolox/tracker /kaggle/working/YOLOX/yolox

In [1]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip install -e .


Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007 (from 1)
Receiving objects: 100% (2007/2007), 79.60 MiB | 42.70 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
/kaggle/working/ByteTrack
Obtaining file:///kaggle/working/ByteTrack
  Preparing metadata (setup.py) ... done
  Running setup.py develop for yolox


In [18]:
print(model)


YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(64, 32, kernel_siz

In [22]:
import os
import cv2
import torch
import numpy as np
from yolox.exp import get_exp
#from yolox.utils.boxes import postprocess
from yolox.tracker.byte_tracker import BYTETracker

import torch
import torchvision

def postprocess(prediction, num_classes=1, conf_thre=0.25, nms_thre=0.45):
    """Applies confidence thresholding and Non-Maximum Suppression (NMS)."""
    
    box_corner = prediction.new(prediction.shape)
    box_corner[:, :, 0] = prediction[:, :, 0] - prediction[:, :, 2] / 2
    box_corner[:, :, 1] = prediction[:, :, 1] - prediction[:, :, 3] / 2
    box_corner[:, :, 2] = prediction[:, :, 0] + prediction[:, :, 2] / 2
    box_corner[:, :, 3] = prediction[:, :, 1] + prediction[:, :, 3] / 2
    prediction[:, :, :4] = box_corner[:, :, :4]

    output = [None for _ in range(len(prediction))]
    
    for i, image_pred in enumerate(prediction):  
        class_conf, class_pred = torch.max(image_pred[:, 5: 5 + num_classes], 1, keepdim=True)

        conf_mask = (image_pred[:, 4] * class_conf.squeeze() >= conf_thre).squeeze()
        detections = torch.cat((image_pred[:, :5], class_conf, class_pred.float()), 1)
        detections = detections[conf_mask]

        if not detections.size(0):
            continue
        
        nms_out_index = torchvision.ops.nms(detections[:, :4], detections[:, 4] * detections[:, 5], nms_thre)
        output[i] = detections[nms_out_index]

    return output


# ✅ Load YOLOX model
exp = get_exp("exps/default/yolox_s.py", "yolox_s")
model = exp.get_model()
ckpt = torch.load("/kaggle/input/yoloxs/yolox_s.pth", map_location="cpu")
model.load_state_dict(ckpt["model"], strict=False)
model.eval()

# ✅ Initialize ByteTrack tracker
class TrackerArgs:
    track_thresh = 0.5
    track_buffer = 30
    match_thresh = 0.8
    aspect_ratio_thresh = 1.6
    min_box_area = 10

tracker = BYTETracker(TrackerArgs())

# ✅ Paths
test_images_path = "/kaggle/input/surveillance-for-retail-stores/tracking/test/01/img1"
mot_output_path = "/kaggle/working/tracking_results.txt"

# ✅ Tracking results list
tracking_results = []

# ✅ Function to preprocess image manually
def preprocess_image(img, target_size=(640, 640)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, target_size)  # Resize
    img = img / 255.0  # Normalize
    img = np.transpose(img, (2, 0, 1))  # HWC → CHW
    img = np.expand_dims(img, axis=0)  # Add batch dim
    return torch.tensor(img, dtype=torch.float32)

# ✅ Process test frames
for img_name in sorted(os.listdir(test_images_path)):
    frame_number = int(img_name.split(".")[0])
    img_path = os.path.join(test_images_path, img_name)
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}, skipping.")
        continue
    
    img_tensor = preprocess_image(img)

    with torch.no_grad():
        outputs = model(img_tensor)
        #print(f"Model Output Shape: {outputs[0].shape if len(outputs) > 0 else 'No output'}")

        if len(outputs) == 0 or outputs[0].ndim != 3:
            continue

        detections = postprocess(outputs[0], num_classes=1, conf_thre=0.25, nms_thre=0.45)
        print(f"Detections Output: {detections}")  # Debugging


    if detections is not None:
        tracked_objects = tracker.update(detections.cpu().numpy(), target_size, 1.0)
        
        for track in tracked_objects:
            track_id = int(track.track_id)
            x, y, w, h = map(int, track.tlwh)
            conf = round(track.score, 4)

            tracking_results.append([frame_number, track_id, x, y, w, h, conf, 1, 1])

# ✅ Sort by (Track ID, Frame Number)
tracking_results.sort(key=lambda x: (x[1], x[0]))

# ✅ Save results in MOT format
with open(mot_output_path, "w") as f:
    for result in tracking_results:
        f.write(",".join(map(str, result)) + "\n")

print(f"✅ MOT format tracking results saved to {mot_output_path}")


<ipython-input-22-4f6d847ffe9a>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/yoloxs/yolox_s.pth", map_location="cpu")


✅ MOT format tracking results saved to /kaggle/working/tracking_results.txt


In [9]:
!pip install cython_bbox faiss-gpu

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.5 MB/s eta 0:00:00:00:0100:01
  Created wheel for cython_bbox: filename=cython_bbox-0.1.5-cp310-cp310-linux_x86_64.whl size=99108 sha256=e7671f02dbfaa610cc44e89db855d4d8aa2c216b55ee960182719196a5694ca2
  Stored in directory: /root/.cache/pip/wheels/c0/b7/68/bab98b7180cda501101a57fb7d36884218ad45ec60c27cd679
Successfully built cython_bbox


In [7]:
!pip install lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.6 MB/s eta 0:00:0000:010:01


In [3]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.1 MB/s eta 0:00:00


In [5]:
!pip install thop

In [4]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from ultralytics import YOLO
#import motmetrics as mm  # New import for MOTA calculation

def enhance_tracking_performance(input_path, output_path):
    # Load the YOLO model with more robust tracking
    model2 = YOLO('/kaggle/input/trackingyolo/pytorch/default/6/best.pt')
    
    # Enhanced tracking parameters
    tracking_config = {
        'iou_threshold': 0.5,  # Adaptive IOU threshold
        'conf_threshold': 0.3,  # Confidence threshold with some wiggle room
        'max_age': 30,  # Maximum frames to keep a track alive without detection
        'min_hits': 3,  # Minimum number of hits to consider a track valid
    }
    
    # Additional tracking enhancement techniques
    tracking_results = []
    track_history = {}
    occlusion_tracker = {}
    
    # Load ground truth for MOTA calculation
    #gt_data = pd.read_csv(gt_path, header=None, names=['frame', 'id', 'x', 'y', 'w', 'h', 'conf', 'class', 'visibility'])
    
    for img_name in sorted(os.listdir(input_path)):
        frame_number = int(img_name.split(".")[0])
        img_path = os.path.join(input_path, img_name)
        img = cv2.imread(img_path)
        
        if img is None:
            continue
        
        # Resize and preprocess image
        img_resized = cv2.resize(img, (704, 704))
        
        # Advanced tracking with more robust parameters
        results = model2.track(
            img_resized, 
            persist=True, 
            iou=tracking_config['iou_threshold'], 
            conf=tracking_config['conf_threshold']
        )[0]
        
        for track in results.boxes:
            # Enhanced track ID and confidence handling
            track_id = int(track.id[0].item()) if track.id is not None and len(track.id) > 0 else -1
            conf = track.conf[0].item()
            cls = int(track.cls[0].item())
            
            if cls == 0:  # Person class
                # Coordinate scaling and conversion
                x_center, y_center, w, h = track.xywh[0].cpu().numpy()
                scale_x = 1920 / 704
                scale_y = 1080 / 704
                
                x_center *= scale_x
                y_center *= scale_y
                w *= scale_x
                h *= scale_y
                
                x = int(x_center - w / 2)
                y = int(y_center - h / 2)
                w, h = int(w), int(h)
                
                # Occlusion and ID stability handling
                if track_id in track_history:
                    prev_x, prev_y = track_history[track_id]
                    distance = np.sqrt((x_center - prev_x)**2 + (y_center - prev_y)**2)
                    
                    # Adaptive ID management
                    if distance > 100:  # Large movement might indicate track switch
                        track_id = max(track_history.keys()) + 1 if track_history else 1
                
                track_history[track_id] = (x_center, y_center)
                
                # Robust tracking result storage
                tracking_results.append([
                    frame_number, track_id, x, y, w, h, 
                    round(conf, 4), 1, 1
                ])
    
    # Post-processing for track stability
    track_id_data = {}
    for row in tracking_results:
        frame_number, track_id = row[0], row[1]
        if track_id not in track_id_data:
            track_id_data[track_id] = []
        track_id_data[track_id].append(row)
    
    # Remove very short tracks
    track_id_data = {k: v for k, v in track_id_data.items() if len(v) >= 20}
    
    # Save processed tracking results
    with open(output_path, "w") as f:
        for track_id in sorted(track_id_data.keys()):
            for row in sorted(track_id_data[track_id], key=lambda x: x[0]):
                f.write(",".join(map(str, row)) + "\n")
    
    

# Usage
enhance_tracking_performance(
    "/kaggle/input/surveillance-for-retail-stores/tracking/test/01/img1", 
    "/kaggle/working/mot_res.txt"
)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.5s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 704x704 39 persons, 43.2ms
Speed: 8.2ms preprocess, 43.2ms inference, 298.1ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 38 persons, 43.1ms
Speed: 3.1ms preprocess, 43.1ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 36 persons, 43.1ms
Speed: 2.0ms preprocess, 43.1ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 36 persons, 43.1ms
Speed: 1.6ms preprocess, 43.1ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 33 persons, 39.6ms
Speed: 2.2ms preprocess, 39.6ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 704)

0: 704x704 37 persons, 39.7ms
Spe

In [ ]:
import motmetrics as mm
import numpy as np

# Load ground truth and predicted tracks
def load_tracks(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    return data

gt_file = '/kaggle/input/filteredgt/filtered_gt.txt'
pred_file = '/kaggle/working/bytetracking_ftlast_results.txt'

gt_data = load_tracks(gt_file)
pred_data = load_tracks(pred_file)

# Create an accumulator to store results
acc = mm.MOTAccumulator(auto_id=True)

# Iterate through each frame
for frame in np.unique(gt_data[:, 0]):
    # Extract ground truth and predictions for the current frame
    gt_objects = gt_data[gt_data[:, 0] == frame]
    pred_objects = pred_data[pred_data[:, 0] == frame]

    # Extract bounding boxes and IDs
    gt_boxes = gt_objects[:, 2:6]  # x, y, width, height
    gt_ids = gt_objects[:, 1]

    pred_boxes = pred_objects[:, 2:6]
    pred_ids = pred_objects[:, 1]

    # Compute pairwise distances (e.g., IoU)
    dist_matrix = mm.distances.iou_matrix(gt_boxes, pred_boxes, max_iou=0.5)

    # Update the accumulator
    acc.update(gt_ids, pred_ids, dist_matrix)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['mota', 'motp', 'idf1', 'precision', 'recall'], name='acc')

# Print the results
print(summary)
